In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1_csky_bugfix_template_flux/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20, ana=ana):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'fermibubbles_50TeV': get_gp_tr('fermibubbles', cutoff=50),
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    if os.path.exists(f_path):
        res_dict[key] = np.load(f_path)
    else:
        print('File does not exist: {}'.format(f_path))

#### Print best fit fluxes

In [ ]:
res_dict

In [ ]:

dNdE = tr_dict['pi0'].to_dNdE(ns=res_dict['pi0'][1], E0=1e5)
E2dNdE = tr_dict['pi0'].to_E2dNdE(ns=res_dict['pi0'][1], E0=100, unit=1e3)
print(dNdE, E2dNdE)


#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if 'fermibubbles' in key: continue
    if key in bkg_file_dict:
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
    
    else:
        print('Running background trials for template {}'.format(key))
        bkg_dict[key] = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
        

In [ ]:
for k, values in bkg_dict.items():
    print(k, len(values))

#### Plot ts distribution

In [ ]:
for key, bg in bkg_dict.items():
    bg_tsd = cy.dists.TSD(bg)
    fig, ax = plot_bkg_trials(bg_tsd)
    ts = res_dict[key][0]
    ns = res_dict[key][1]
    ax.axvline(
        ts, color='0.8', ls='--', lw=2,
        label='TS: {:3.3f} | ns: {:3.1f}'.format(ts, ns), 
    )
    ts_5sig = bg_tsd.isf_nsigma(5)
    ax.axvline(
        ts_5sig, ls='--', lw=1,
        label='5-sigma TS: {:3.3f}'.format(ts_5sig), 
    )
    ax.set_title('Template: {}'.format(key))
    ax.set_yscale('log')
    ax.legend()
    fig.savefig('{}/ts_dist_{}.png'.format(plot_dir, key))

#### Compute Significance

In [ ]:
p_val_dict = {}
sigma_dict = {}
max_n = 3000000
for key, bg in bkg_dict.items():
    print(key)
    bg_tsd = cy.dists.TSD(bg[:max_n])
    p_val_dict[key] = bg_tsd.sf(bg[:max_n])
    sigma_dict[key] = bg_tsd.sf_nsigma(bg[:max_n])


#### Plot Trial Correlation

In [ ]:
mask = np.zeros_like(bkg_dict['pi0'][:max_n])
sigma_threshold = 0.5

for key, tr in sigma_dict.items():

    mask = np.logical_or(mask, sigma_dict[key] > sigma_threshold)
    

In [ ]:
import matplotlib as mpl

def plot_corr_ax(ax, key1, key2, mask=None, norm=None):
    
    if mask is None:
        mask = np.ones_like(sigma_dict[key1], dtype=bool)
        
    ax.hist2d(
        sigma_dict[key1][mask], sigma_dict[key2][mask],
        bins=bins, norm=norm, cmin=1,
    )
    ax.plot(
        (bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), 
        ls='--', color='0.7', lw=3,
    )
    ax.set_xlabel('$n\cdot \sigma$ of {}'.format(key1))
    ax.set_ylabel('$n\cdot \sigma$ of {}'.format(key2))

fig, axes = plt.subplots(3, 1, figsize=(9, 9))

bins = (np.linspace(0, 6, 50), np.linspace(0, 6, 50))
norm = mpl.colors.LogNorm(vmin=1, vmax=1e5)
mask = None
plot_corr_ax(axes[0], 'pi0', 'kra5', mask=mask, norm=norm)
plot_corr_ax(axes[1], 'kra5', 'kra50', mask=mask, norm=norm)
plot_corr_ax(axes[2], 'pi0', 'kra50', mask=mask, norm=norm)
fig.tight_layout()
fig.savefig('{}/gp_trial_correlation.png'.format(plot_dir))


In [ ]:
corr_keys = ['pi0', 'kra5', 'kra50']

max_nsigma = np.max(
    np.stack([sigma_dict[k] for k in corr_keys]),
    axis=0,
)
bg_max = cy.dists.TSD(max_nsigma)

In [ ]:
from scipy import stats

nsigma_chosen = res_dict['pi0'][3]# 4.705
pval_chosen = stats.norm.sf(nsigma_chosen)
nsigma_corrected = bg_max.sf_nsigma(nsigma_chosen)

fig, ax = plt.subplots()
ax.hist(max_nsigma, bins=np.linspace(0, 6, 200), label='Correlated bkg trials')
ax.set_xlabel('Max n-sigma')
ax.set_ylabel('Number of trials')
ax.set_yscale('log')
ax.axvline(
    nsigma_chosen, ls='--', color='0.7', 
    label='Unblinded: {:3.3f}$\sigma$ | Corrected: {:3.3f}$\sigma$'.format(
        nsigma_chosen, nsigma_corrected),
)
ax.legend(loc='upper right')
fig.savefig('{}/gp_trial_correction_hist.png'.format(plot_dir))

pval_corrected = bg_max.sf(nsigma_chosen)
print('Correcting for: {}'.format(corr_keys))
print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Pre-trial p-value of: {}'.format(pval_chosen))
print('Post-trial correlated n-sigma: {} | factor: {}'.format(nsigma_corrected, pval_corrected/pval_chosen))
print('Post-trial correlated p-value: {} | factor: {}'.format(pval_corrected, pval_corrected/pval_chosen))
print('Post-trial conservative n-sigma: {} | factor: {}'.format(stats.norm.isf(pval_chosen * len(corr_keys)), len(corr_keys)))
print('Post-trial conservative p-value: {} | factor: {}'.format(pval_chosen * len(corr_keys), pval_chosen * len(corr_keys)/pval_chosen))



In [ ]:
nsigma_chosen = res_dict['pi0'][3]# 4.705
pval_chosen = stats.norm.sf(nsigma_chosen)
pval_conservative = pval_chosen * 3
nsigma_conservative = stats.norm.isf(pval_conservative)

print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Post-trial conservative: {} | factor: {}'.format(nsigma_conservative, pval_conservative/pval_chosen))


## Load and Plot Skymaps

In [ ]:
ss_results = np.load(
    os.path.join(
        cg.base_dir, 'skyscan/results/unblinded_skyscan.npy'),
    allow_pickle=True,
)[()]
ss_trial = ss_results['ss_trial']




In [ ]:
ss_results


In [ ]:
names = ['mlog10p', 'ts', 'ns', 'gamma']
for loc in ['south', 'north']:
    print('Hottest spot in {}:'.format(loc))
    for i, name in enumerate(names):
        print('  {}: {}'.format(name, ss_trial[i,ss_results['ipix_max_{}'.format(loc)]]))
    print('  {}: {}'.format('p-value',  10**-ss_trial[0, ss_results['ipix_max_{}'.format(loc)]]))


#### Compare against pre-patch standard candle version

In [ ]:
ss_results__candle = np.load(
    os.path.join('/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/skyscan/results/unblinded_skyscan.npy'),
    allow_pickle=True,
)[()]
ss_trial__candle = ss_results__candle['ss_trial']

names = ['mlog10p', 'ts', 'ns', 'gamma']
for i, key in enumerate(names):
    print('{} all close: {}'.format(
        key, np.allclose(ss_trial__candle[i], ss_trial[i])))

print(np.max(ss_trial__candle[2] - ss_trial[2]))
print(ss_results['ipix_max_north'], ss_results__candle['ipix_max_north'])

max_ipix_old = ss_results__candle['ipix_max_north']
max_ipix_new = ss_results['ipix_max_north']

print('\nComparing old and new northern hotspot on patch 0:')
for i, key in enumerate(names):
    print('  {:>7}: {:1.4e} [old] | {:1.4e} [new]'.format(
        key, ss_trial__candle[i, max_ipix_old], ss_trial__candle[i, max_ipix_new]))

print('\nComparing old and new northern hotspot on patch 1:')
for i, key in enumerate(names):
    print('  {:>7}: {:1.4e} [old] | {:1.4e} [new]'.format(
        key, ss_trial[i, max_ipix_old], ss_trial[i, max_ipix_new]))
    
print()

In [ ]:
10**-3.4003e+00, 10**-3.4067e+00

#### Define catalogs

In [ ]:
# https://www.nasa.gov/mission_pages/GLAST/news/gammaray_best.html
# https://www.nasa.gov/images/content/317870main_Fermi_3_month_labeled_new.jpg
fermi_sources = {
    # ra, dec
    'NGC 1275': (049.9506656698585, +41.5116983765094),
    '3C 454.3': (343.49061658, +16.14821142),
    '47 Tuc': (006.022329, -72.081444),
    '0FGL J1813.5-1248': (273.349033, -12.766842),
    '0FGL J0614.3-3330': (093.5431162, -33.4983656),
    'PKS 0727-115': (112.57963530917, -11.68683347528),
    'Vela': (128.5000, -45.8333),
    'Geminga': (098.475638, +17.770253),
    'Crab': (083.63308, +22.01450),
    'LSI +61 303': (040.1319341179735, +61.2293308716971),
    'PSR J1836+5925': (279.056921, +59.424936),
    'PKS 1502+106': (226.10408242258, +10.49422183753),
    #'Cygnus X-3': (308.10742, +40.95775), # not one of the top Fermi sources
}

cat_dict = {}
for cat_str in ['pwn', 'snr', 'unid']:
    catalog_file = os.path.join(
        cg.catalog_dir, '{}_ESTES_12.pickle'.format(cat_str))
    cat_dict[cat_str] = np.load(catalog_file, allow_pickle=True)

src_list_file = os.path.join(cg.catalog_dir, 'Source_List_DNNC.pickle')
sourcelist = np.load(src_list_file, allow_pickle=True)
sourcelist['ra_deg'] = sourcelist['RA']
sourcelist['dec_deg'] = sourcelist['DEC']
cat_dict['Source List'] = sourcelist

#### Plot template contours

In [ ]:
class ContourSkymap:
    def __init__(self, skymap, nside=None):
        
        # upscale skymap
        if nside is not None:
            skymap = hp.ud_grade(skymap, nside_out=nside)
        else:
            nside = hp.get_nside(skymap)
        
        # normalize such that sum of pixel values equals one
        self.prob_values = skymap / np.sum(skymap)
        self.neg_llh_values = -np.log10(self.prob_values)
        self.nside = hp.get_nside(self.prob_values)
        self.npix = hp.nside2npix(self.nside)
        
        self.theta, self.phi = self.get_healpix_grid()
        
        # sort healpix points according to neg llh
        sorted_indices = np.argsort(self.neg_llh_values)
        self.theta_s = self.theta[sorted_indices]
        self.phi_s = self.phi[sorted_indices]
        self.neg_llh_values_s = self.neg_llh_values[sorted_indices]
        self.prob_values_s = self.prob_values[sorted_indices]
    
        self.cdf_values_s = np.cumsum(self.prob_values_s)
    
    def get_healpix_grid(self):
        npix = hp.nside2npix(self.nside)
        theta, phi = hp.pix2ang(self.nside, np.r_[:npix])
        return theta, phi
    
    def quantile_to_pdf_value(self, quantile):
        """Get pixel probability value
        """
        assert quantile >= 0., quantile
        assert quantile <= 1., quantile

        index = np.searchsorted(self.cdf_values_s, quantile)
        return self.prob_values_s[index]
    
    def _get_level_indices(self, level=0.5, delta=0.01):
        """Get indices of healpix map, which belong to the specified
        contour as defined by: level +- delta.

        Parameters
        ----------
        level : float, optional
            The contour level. Example: a level of 0.7 means that 70% of events
            are within this contour.
        delta : float, optional
            The contour is provided by selecting directions from the sampled
            ones which have cdf values within [level - delta, level + delta].
            The smaller delta, the more accurate the contour will be. However,
            the number of available sample points for the contour will also
            decrease.

        Returns
        -------
        int, int
            The starting and stopping index for a slice of sampled events
            that lie within the contour [level - delta, level + delta].

        Raises
        ------
        ValueError
            If number of resulting samples is too low.
        """
        assert level >= 0., level
        assert level <= 1., level

        index_min = np.searchsorted(self.cdf_values_s, level - delta)
        index_max = min(self.npix,
                        np.searchsorted(self.cdf_values_s, level + delta))

        if index_max - index_min <= 10:
            raise ValueError('Number of samples is too low!')

        return index_min, index_max
    
    def contour(self, level=0.5, delta=0.01):
        """Get zenith/azimuth paris of points that lie with the specified
        contour [level - delta, level + delta].

        Parameters
        ----------
        level : float, optional
            The contour level. Example: a level of 0.7 means that 70% of events
            are within this contour.
        delta : float, optional
            The contour is provided by selecting directions from the sampled
            ones which have cdf values within [level - delta, level + delta].
            The smaller delta, the more accurate the contour will be. However,
            the number of available sample points for the contour will also
            decrease.

        Returns
        -------
        np.array, np.array
            The theta/phi pairs that lie within the contour
            [level - delta, level + delta].
        """
        index_min, index_max = self._get_level_indices(level, delta)
        return (self.theta_s[index_min:index_max],
                self.phi_s[index_min:index_max])

In [ ]:
def get_smeared_template(key, smearing=5):
    space_pdf = tr_dict[key].llh_models[0].pdf_ratio_model.models[0]
    sigma_idx = np.searchsorted(np.rad2deg(space_pdf.sigmas), smearing)
    template_smeared = space_pdf.pdf_space_sig[sigma_idx]
    return np.array(template_smeared)


#### Skymap Plotting Class

In [ ]:
import utils
import contour_compute


class SkymapPlotter:
    
    def __init__(self, fermi_sources=fermi_sources, cat_dict=cat_dict, ss_results=ss_results, **kwargs):
        self.cat_dict = cat_dict
        self.fermi_sources = fermi_sources
        self.ss_results = ss_results
        self.coord = None
        
        self.fig, self.ax, self.sp, self.cb, self.pc = self.plot_skymap(**kwargs)
    
    def add_skymap_layer(self, m, ax=None, input_coord='C', rasterized=True, **kw):
        
        if input_coord == 'C' and self.sp.coord == 'G':
            m = SkymapPlotter.equatorial_to_galactic(m)
                
        if ax is None:
            ax = self.ax
        lat, lon, Z = self.sp.map_to_latlonz(m)
        pc = ax.pcolormesh(lon, lat, Z, rasterized=rasterized, **kw)
        return lat, lon, Z, pc
        
    @staticmethod
    def plot_skymap(
                skymap, fig=None, ax=None, outfile=None, figsize=(9, 6),
                vmin=None, vmax=None, label=None, norm=None,
                cmap=cy.plotting.skymap_cmap,
                input_coord='C',
                n_cb_ticks=4,
                gp_kw=dict(color='.3', alpha=0.5), gp_lw=1.,
                plot_gp=True, annotate=True,
                rasterized=True,
                cb_kw_direct={},
                **kwargs
            ):
        """Plot a skymap

        Parameters
        ----------
        skymap : array_like
            The skymap to plot.
        outfile : str, optional
            The output file path to which to plot if provided.
        vmin : float, optional
            The minimum value for the colorbar.
        vmax : float, optional
            The maximum value for the colorbar.
        figsize : tuple, optional
            The figure size to use.
        label : str, optional
            The label for the colorbar.

        Returns
        -------
        fig, ax
            The matplotlib figure and axis.
        """
        if fig is None:
            fig, ax = plt.subplots(
                subplot_kw=dict(projection='aitoff'), figsize=figsize)

        if 'coord' in kwargs and kwargs['coord'] == 'G':
            nohr = True
            if input_coord == 'C':
                skymap = SkymapPlotter.equatorial_to_galactic(skymap)
        else:
            nohr = False

        sp = cy.plotting.SkyPlotter(
            pc_kw=dict(cmap=cmap, vmin=vmin, vmax=vmax, norm=norm, rasterized=rasterized), 
            **kwargs
        )
        pc, cb = sp.plot_map(
            ax, skymap, n_ticks=n_cb_ticks, nohr=nohr, pc_kw={'rasterized': rasterized}, cb_kw=cb_kw_direct)
    
        SkymapPlotter.annotate_skymap(
            ax=ax, sp=sp, annotate=annotate, plot_gp=plot_gp, gp_kw=gp_kw, gp_lw=gp_lw,
        )
        if False:
            if sp.coord == 'G' and annotate:
                kw = dict(xycoords='axes fraction', textcoords='offset pixels', verticalalignment='center')
                ax.annotate(r'l = -180°', xy=(1, .5), xytext=(10, 0), horizontalalignment='left', **kw)
                ax.annotate(r'l = 180°', xy=(0, .5), xytext=(-10, 0), horizontalalignment='right', **kw)

            if sp.coord != 'G' and plot_gp:
                sp.plot_gp(ax, lw=gp_lw, rasterized=rasterized, **gp_kw)
                sp.plot_gc(ax, **gp_kw)
            kw = dict(color='.5', alpha=.5)
            ax.grid(**kw)
        cb.set_label(label)
        fig.tight_layout()
        if outfile is not None:
            fig.savefig(outfile)

        return fig, ax, sp, cb, pc
    
    @staticmethod
    def annotate_skymap(ax, sp, annotate=True, plot_gp=True, gp_kw=dict(color='.3', alpha=0.5, rasterized=True), gp_lw=1.):
        if sp.coord == 'G' and annotate:
            kw = dict(xycoords='axes fraction', textcoords='offset pixels', verticalalignment='center')
            ax.annotate(r'l = -180°', xy=(1, .5), xytext=(10, 0), horizontalalignment='left', **kw)
            ax.annotate(r'l = 180°', xy=(0, .5), xytext=(-10, 0), horizontalalignment='right', **kw)
        
        if sp.coord != 'G' and plot_gp:
            sp.plot_gp(ax, lw=gp_lw, **gp_kw)
            sp.plot_gc(ax, **gp_kw)
        kw = dict(color='.5', alpha=.5)
        ax.grid(**kw)
    
    @staticmethod
    def equatorial_to_galactic(m, rot=180):
        r = hp.Rotator(rot=rot, coord='CG')
        return r.rotate_map_pixel(m)

    @staticmethod
    def equatorial_to_galactic_coords(theta, phi, rot=180):
        r = hp.Rotator(rot=rot, coord='CG')
        return r(theta, phi)
    
    @staticmethod
    def galactic_to_equatorial(m, rot=180):
        r = hp.Rotator(rot=rot, coord='CG', inv=True)
        return r.rotate_map_pixel(m)

    @staticmethod
    def galactic_to_equatorial_coords(theta, phi, rot=180):
        r = hp.Rotator(rot=rot, coord='CG', inv=True)
        return r(theta, phi)
    
    def convert_theta_phi_to_mpl_coords(self, theta, phi, convert=True):
        if self.sp.coord == 'G' and convert:
            theta, phi = self.equatorial_to_galactic_coords(theta, phi)
        x, y = self.sp.thetaphi_to_mpl(theta, phi)
        return x, y
        
    def convert_ra_dec_to_mpl_coords(self, ra, dec):
        theta = np.pi/2. - dec
        phi = ra
        return self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)

    def draw_equator(self, ax=None, color='0.6', s=1, **kwargs):
        if ax is None:
            ax = self.ax
        phi = np.linspace(0., 2*np.pi, 10000)
        dec = np.zeros_like(phi)
        x, y = self.convert_ra_dec_to_mpl_coords(ra=phi, dec=dec)
        return ax.scatter(x, y, marker='.', color=color, s=s)

    def plot_catalog(self, ax=None, sp=None, marker='x', color='red', keys=None, labels=None, cat_dict=None, **kwargs):
        
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
        
        if cat_dict is None:
            cat_dict = self.cat_dict
            
        if keys is None:
            keys = list(cat_dict.keys())
        if labels is None:
            labels = keys
            
        for cat_str, label in zip(keys, labels):
            cat = cat_dict[cat_str]
            x, y = self.convert_ra_dec_to_mpl_coords(
                ra=np.deg2rad(cat.ra_deg), dec=np.deg2rad(cat.dec_deg))
            ax.scatter(x, y, marker=marker, color=color, label=label, **kwargs)
    
    def plot_hotspots(self, ax=None, sp=None, marker='x', color='0.8', **kwargs):
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
            
        # plot hottest spots
        for res_str in ['ipix_max_north', 'ipix_max_south']:
            theta, phi = hp.pix2ang(128, self.ss_results[res_str])
            x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)
            ax.scatter(x, y, marker=marker, color=color, **kwargs)
    
    def plot_fermi_sources(self, ax=None, sp=None, marker='+', color='1.0', sources=None):
        
        if sources is None:
            sources = self.fermi_sources
        
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
            
        for key, (ra_deg, dec_deg) in sources.items():
            x, y = self.convert_ra_dec_to_mpl_coords(
                ra=np.deg2rad(ra_deg), dec=np.deg2rad(dec_deg))
            ax.scatter(x, y, marker=marker, color=color)
    
    def get_contour(self, skymap, quantiles=[0.5], geodesic='planar'):
        """Get contours

        Returns
        -------
        contours_by_level : list(list(list(point)))
            The contours for each level
            Outermost list indexes by level
            Second list indexes by contours at a particular level
            Third list indexes by points in each contour
            Points are of the same form as sample_points
        """
        nside = hp.get_nside(skymap)
        theta, phi = hp.pix2ang(nside=nside, ipix=np.arange(hp.nside2npix(nside)))
        
        if self.sp.coord == 'G':
            skymap = SkymapPlotter.equatorial_to_galactic(skymap)
        
        # compute PDF levels for provided quantiles 
        contour_map = ContourSkymap(skymap=skymap)

        levels = []
        for quantile in quantiles:
            levels.append(contour_map.quantile_to_pdf_value(quantile))
        
        # compute sample points in which to compute the contours
        if geodesic == 'spherical':
            sample_points = np.stack((theta, phi), axis=1)
        elif geodesic == 'planar':
            x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi, convert=False)
            sample_points = np.stack((x, y), axis=1)
        else:
            raise ValueError('Unknown geodesic: {}'.format(geodesic))

        contours = contour_compute.compute_contours(
            sample_points=sample_points, samples=contour_map.prob_values, levels=levels, geodesic=geodesic)
        return contours

    def plot_template_contour(
                self, template_str, smearing_deg=5., quantiles=[0.5], ls=['-'], color=[None], 
                geodesic='planar', kwargs_list=[{}, {}],
                template=None,
        ):
        assert len(quantiles) == len(ls)
        assert len(quantiles) == len(color)
        
        if template is None:
            template = get_smeared_template(template_str, smearing=smearing_deg)
        contours = self.get_contour(skymap=template, quantiles=quantiles, geodesic=geodesic)
        for ls_i, color_i, kwargs_i, contour in zip(ls, color, kwargs_list, contours):
            for contour_i in contour:
                if geodesic == 'spherical':
                    x, y = self.convert_theta_phi_to_mpl_coords(theta=contour_i[:, 0], phi=contour_i[:, 1], convert=False)
                elif geodesic == 'planar':
                    x, y = contour_i[:, 0], contour_i[:, 1]
                else:
                    raise ValueError('Unknown geodesic: {}'.format(geodesic))
                if len(x) > 2:
                    self.ax.plot(x, y, ls=ls_i, color=color_i, **kwargs_i)
        return contours
    
    def plot_template_contour_points(
                self, template_str, level, 
                smearing_deg=5, color='0.2', delta=0.01, 
                marker='.', s=1,
                ax=None, sp=None,
            ):
        contour_map = ContourSkymap(get_smeared_template(template_str, smearing=smearing_deg))
        theta, phi = contour_map.contour(level=level, delta=delta)
        x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)
        self.ax.scatter(x, y, marker=marker, s=s, color=color)


#### Plot Gamma

In [ ]:
cmap = plt.cm.viridis  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(1, 4, 17)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

gamma_masked = np.array(ss_trial[3])
#gamma_masked[10**(-ss_trial[0]) > 0.5] = np.nan


skymap_plotter = SkymapPlotter(skymap=gamma_masked, cmap=cmap, norm=norm, label=r'$\gamma$')

#skymap_plotter.plot_catalog()
skymap_plotter.plot_hotspots()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.tight_layout()
skymap_plotter.fig.savefig('{}/skymap_gamma.png'.format(plot_dir))


#### Plot Gamma Alpha in Equatorial and Galactic Coordinates

In [ ]:
def alpha_func(nsigma):
    cutoff = -0.52
    x = (nsigma - cutoff) / (2.2 - cutoff)
    x = x**4
    x[nsigma < cutoff] = 0.
    x += 0.02
    x = np.round(x, 1)
    alpha = np.clip(x, 0., 1.)
    return alpha

x = np.linspace(-4, 4, 100)
fig, ax = plt.subplots()
ax.plot(x, alpha_func(x))
ax.set_xlabel('$n \cdot \sigma$')
ax.set_ylabel('Alpha Value')
fig.savefig('{}/alpha_mapping.png'.format(plot_dir))


In [ ]:
from scipy import stats

# original settings:
# n_gamma_bins = 31
# cmap=plt.cm.ocean, color_range=np.r_[0:200]
n_gamma_bins = 13


def get_gamma_cmap_and_norm(cmap=plt.cm.ocean, color_range=np.r_[0:190][::-1], n_gamma_bins=n_gamma_bins):
    if color_range is None:
        color_range = list(range(cmap.N))
        
    cmaplist = [cmap(i) for i in color_range]
    # create the new map
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)
    #cmap = mpl.colors.ListedColormap(cmaplist)
    # define the bins and normalize
    bounds = np.linspace(1, 4, n_gamma_bins)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    return cmap, norm

def alpha_to_white(colors):
    colors = np.atleast_2d(colors)
    alpha = colors[..., -1:]
    color = colors[..., :-1]
    white = np.ones_like(color)
    return alpha*color + (1 - alpha)*white

def get_facecolors(gammas, nsigmas, cmap_norm_func=get_gamma_cmap_and_norm, alpha_func=alpha_func):
    
    cmap, norm = cmap_norm_func()
    colors = np.array(cmap(norm(gammas)))
    colors[..., 3] = alpha_func(nsigma=nsigmas)
    colors = alpha_to_white(colors)
    colors = np.concatenate([colors, np.ones((len(colors), 1))], axis=-1)
    return colors  

class Encoded2DColorMap(mpl.colors.Colormap):
    def __call__(self, X, alpha=None, bytes=False):
        if bytes:
            raise NotImplementedError
        if alpha is not None:
            raise NotImplementedError
            
        if np.any(X > 1.):
            X = np.array(X) / self.N
            raise NotImplementedError

        mask_bad = X.mask if np.ma.is_masked(X) else np.isnan(X)  # Mask nan's.
        xa = np.array(X, copy=True)
        xa[mask_bad] = self._i_bad

        # decode into gamma and nsigma
        gammas, nsigmas = Encoded2DColorMap.decode(X)

        # get colors based on gamma and nsigma
        colors = get_facecolors(gammas, nsigmas)
        colors[mask_bad] = (0., 0., 0., 0.)
        #return cmap(norm(gammas))
        return colors

    @staticmethod
    def encode(gamma, nsigma):
        n_dec = 3
        min_nsigma = -4

        assert np.all(nsigma >= min_nsigma)
        assert np.all(nsigma < 10. + min_nsigma)

        # discretize gamma
        gamma_disc = np.round(gamma, n_dec) * 10**n_dec
        return (gamma_disc * 10. + nsigma - min_nsigma) / (100. * 10**n_dec)

    @staticmethod
    def decode(z):
        n_dec = 3
        min_nsigma = -4

        norm_factor = (100. * 10**n_dec)
        gamma = (z * norm_factor // 10.) / 10**n_dec
        nsigma = np.mod(z * norm_factor, 10.) + min_nsigma
        return gamma, nsigma

    
def draw_2d_gamma(fig, ax, coord, label=r'$\gamma$', ss_trial=ss_trial, **kwargs):
    cmap, norm = get_gamma_cmap_and_norm()
    nans = np.ones_like(ss_trial[3]) * np.nan
    skymap_plotter = SkymapPlotter(skymap=nans, cmap=cmap, norm=norm, label=label, fig=fig, ax=ax, coord=coord, **kwargs)
    # ------------------------------

    gamma_masked = np.array(ss_trial[3])
    ss_nsigmas = stats.norm.isf(10**-ss_trial[0])
    
    # rotate if necessary
    if coord == 'G':
        gamma_masked = SkymapPlotter.equatorial_to_galactic(gamma_masked)
        ss_nsigmas = SkymapPlotter.equatorial_to_galactic(ss_nsigmas)
    
    # encode Gamma and nsigma in one value
    map_encoded = Encoded2DColorMap.encode(gamma_masked, ss_nsigmas)

    # mask out pole
    mask_pole = gamma_masked == 0.
    map_encoded[mask_pole] = np.nan
        
    skymap_plotter.add_skymap_layer(
        map_encoded,
        input_coord=coord,
        cmap=Encoded2DColorMap(name='Custom2DMap'), 
        norm=mpl.colors.Normalize(vmin=0., vmax=1.),
    )
    return skymap_plotter


def plot_2d_colorbar(ax_cmatrix, n_gamma_bins, flip_labels=False):
    use_nsigma = True
    bins_nsigma = np.linspace(-.01, 3., 11)
    bins_ml10p = -np.log10(stats.norm.sf(bins_nsigma))
    gamma_mesh, nsigma_mesh = np.meshgrid(np.linspace(1, 4, n_gamma_bins), bins_nsigma)
    gamma_mesh, pval_mesh = np.meshgrid(np.linspace(1, 4, n_gamma_bins), bins_ml10p)
    matrix_encoded = Encoded2DColorMap.encode(gamma_mesh, nsigma_mesh)
    if use_nsigma:
        yy_mesh = nsigma_mesh
        ylabel = r'$\sigma$'
    else:
        yy_mesh = pval_mesh
        ylabel = r'$-\log_{10}(p_\mathrm{local})$'
    if flip_labels:
        if use_nsigma:
            ylabel = r'Pre-trial significance / $\sigma$'
        pc = ax_cmatrix.pcolormesh(
            yy_mesh.T, gamma_mesh.T, matrix_encoded.T,
            cmap=Encoded2DColorMap(name='Custom2DMap'), 
            norm=mpl.colors.Normalize(vmin=0., vmax=1.),
        )
        ax_cmatrix.set_ylabel(r'$\gamma$')
        ax_cmatrix.set_xlabel(ylabel)
    else:
        pc = ax_cmatrix.pcolormesh(
            gamma_mesh, yy_mesh, matrix_encoded,
            cmap=Encoded2DColorMap(name='Custom2DMap'), 
            norm=mpl.colors.Normalize(vmin=0., vmax=1.),
        )
        ax_cmatrix.set_xlabel(r'Spectral index $\gamma$')
        ax_cmatrix.set_ylabel(ylabel)

for flip_labels in [True, False]:
    #for coord in ['G', 'C']:
    for coord in ['C']:

        # Define figure layout and axes
        fig = plt.figure(constrained_layout=True, figsize=(9, 6))
        gs = fig.add_gridspec(2, 3, height_ratios=[8, 1], width_ratios=[1, 4, 1])
        ax = fig.add_subplot(gs[0, :], projection='aitoff')
        ax_cmatrix = fig.add_subplot(gs[1, 1])

        # plot 2D gamma skymap
        skymap_plotter = draw_2d_gamma(fig=fig, ax=ax, coord=coord)
        skymap_plotter.cb.remove() 

        # plot cbar matrix
        plot_2d_colorbar(ax_cmatrix=ax_cmatrix, n_gamma_bins=n_gamma_bins, flip_labels=flip_labels)

        # redraw grid and annotations
        skymap_plotter.annotate_skymap(ax=skymap_plotter.ax, sp=skymap_plotter.sp)

        #skymap_plotter.plot_catalog()
        #skymap_plotter.plot_hotspots()
        skymap_plotter.plot_fermi_sources(color='1.')
        skymap_plotter.plot_fermi_sources(sources={'d': fermi_sources['3C 454.3']}, color='0.3')
        
        if flip_labels:
            suffix = '_flipped'
        else:
            suffix = ''

        skymap_plotter.fig.tight_layout()
        if coord == 'C':
            skymap_plotter.fig.savefig('{}/skymap_gamma_alpha{}.png'.format(plot_dir, suffix))
        else:
            skymap_plotter.draw_equator()
            skymap_plotter.fig.savefig('{}/skymap_gamma_galactic_alpha{}.png'.format(plot_dir, suffix))


#### Plot ns

In [ ]:
skymap_plotter = SkymapPlotter(skymap=ss_trial[2], cmap='viridis', label='ns', vmin=0, vmax=None)
#skymap_plotter.plot_catalog()
skymap_plotter.plot_hotspots()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.savefig('{}/skymap_ns.png'.format(plot_dir))
    

#### Cross-check effect of Standard Candle Run

In [ ]:
import pandas as pd
mask = np.logical_or(a.bg_data.run == 125910, a.bg_data.run == 125911)
mask = a.bg_data.run == 125911

print('Run:', a.bg_data.run[mask][:1])
print('event:', a.bg_data.event[mask][:1])
print('energy:', a.bg_data.energy[mask][:1])
print('azimuth:', np.rad2deg(a.bg_data.azimuth[mask][:1]))
print('ra:', np.rad2deg(a.bg_data.ra[mask][:1]))
print('dec:', np.rad2deg(a.bg_data.dec[mask][:1]))

cat_candle = {'candle': pd.DataFrame({
    'dec_deg': np.rad2deg(a.bg_data.dec[mask])[:1],
    'ra_deg': np.rad2deg(a.bg_data.ra[mask])[:1],
})}

# p-value
skymap_plotter = SkymapPlotter(
    skymap=ss_trial[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, 
    gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=7, gp_lw=1.)
skymap_plotter.plot_catalog(marker='x', color='r', cat_dict=cat_candle)
skymap_plotter.ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color='0.3', transform=skymap_plotter.ax.transAxes)
skymap_plotter.fig.savefig('{}/skymap_pvalues_standard_candle.png'.format(plot_dir))

# ns
skymap_plotter = SkymapPlotter(skymap=ss_trial[2], cmap='viridis', label='ns', vmin=0, vmax=None)
skymap_plotter.plot_catalog(marker='x', color='r', cat_dict=cat_candle)
skymap_plotter.fig.savefig('{}/skymap_ns_standard_candle.png'.format(plot_dir))

# -----
# gamma
# -----
cmap = plt.cm.viridis  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(1, 4, 17)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
gamma_masked = np.array(ss_trial[3])
skymap_plotter = SkymapPlotter(skymap=gamma_masked, cmap=cmap, norm=norm, label=r'$\gamma$')
skymap_plotter.plot_catalog(marker='x', color='r', cat_dict=cat_candle)
skymap_plotter.fig.tight_layout()
skymap_plotter.fig.savefig('{}/skymap_gamma_standard_candle.png'.format(plot_dir))



#### Plot p-value plot

In [ ]:
skymap_plotter = SkymapPlotter(
    skymap=ss_trial[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, 
    gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=7, gp_lw=1.)
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
skymap_plotter.ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color='0.3', transform=skymap_plotter.ax.transAxes)

skymap_plotter.fig.savefig('{}/skymap_pvalues.png'.format(plot_dir))


#### Plot N-sigmas (Paper Plot)

In [ ]:
from scipy import stats

# -------------------------------------
# Compute skymap nsigma with boundaries
# -------------------------------------
ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])

min_val = -0.52
mask = ss_nsigmas <= min_val
ss_nsigmas[mask] = min_val

mask_pole = ss_trial[3] == 0.
ss_nsigmas[mask_pole] = np.nan

# -------------
# norm and cmap
# -------------
cmap = plt.cm.RdBu_r  # define the colormap
N_colors = cmap.N // 2
print('N_colors', N_colors)
cmaplist = [cmap(i) for i in range(N_colors, cmap.N)]
cmaplist[0] = (1., 1., 1., 1.) # force first bin to be completely white

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0, 4, 9)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)


skymap_plotter = SkymapPlotter(
    skymap=ss_nsigmas, cmap=cmap, norm=norm, label=r'Pre-trial significance / $\sigma$', 
    gp_kw=dict(color='0.3', alpha=0.5), gp_lw=1., n_cb_ticks=9)
#skymap_plotter.plot_catalog(marker='x', color='k')
#skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
#skymap_plotter.plot_fermi_sources(marker='+', color='k')

skymap_plotter.ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color='0.3', transform=skymap_plotter.ax.transAxes)

# add axis and panel labels
if True:
    skymap_plotter.ax.text(
        -0.05, 0.5,r'Declination $\delta$', 
        rotation=90.,
        ha='right', va='center', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    skymap_plotter.ax.text(
        0.5, -0.06, r'Right Ascension $\alpha$', 
        ha='center', va='bottom', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )

smearing_deg = 13
q = 0.2
#skymap_plotter.plot_template_contour('pi0', quantiles=[q], color=['0.2'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('kra5', quantiles=[q], color=['green'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('pi0', quantiles=[q, 0.5], color=['0.2', '0.5'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour_points('pi0', level=q, color='0.2', smearing_deg=smearing_deg, delta=0.007)
#skymap_plotter.plot_template_contour_points('kra5', level=q, color='green', smearing_deg=smearing_deg, delta=0.004)

#skymap_plotter.fig.savefig('{}/skymap_nsigma_s{:1.1f}_q{:1.1f}.png'.format(plot_dir, smearing_deg, q))
skymap_plotter.fig.savefig('{}/skymap_nsigma.png'.format(plot_dir, smearing_deg, q), dpi=300, bbox_inches='tight')
skymap_plotter.fig.savefig('{}/skymap_nsigma.pdf'.format(plot_dir, smearing_deg, q), dpi=300, bbox_inches='tight')

embargo_str = 'Under Embargo,\nNot For Proceedings'
skymap_plotter.ax.text(
    .0, 0.02, embargo_str, 
    ha='left', va='top', color='red', fontsize=18,
    transform=skymap_plotter.ax.transAxes,
)
if False:
    skymap_plotter.ax.text(
        0.5, 0.75,embargo_str, 
        ha='center', va='bottom', color='0.', fontsize=18,
        transform=skymap_plotter.ax.transAxes,
    )
skymap_plotter.fig.savefig('{}/skymap_nsigma__embargo.png'.format(plot_dir, smearing_deg, q), dpi=300, bbox_inches='tight')
skymap_plotter.fig.savefig('{}/skymap_nsigma__embargo.pdf'.format(plot_dir, smearing_deg, q), dpi=300, bbox_inches='tight')


In [ ]:
plt.hist(stats.norm.isf(10**-ss_trial[0]), bins=np.linspace(-1, 0, 1000))
plt.axvline(-0.52, color='0.7')

#### Combined skymap with insert: Helper-Functions

In [ ]:
import matplotlib
from matplotlib.patches import ConnectionPatch


def adjust_skymap_insert(skymap_plotter_insert, ax_insert, smearing_deg=7, latitude_deg=15, 
                         adjust_cb=True, modify_xticks=True, modify_yticks=True, contour_name='pi0', 
                         contour_qs=[0.2, 0.5], contour_colors=['0.2', '0.5'], contour_ls=['--', '-.'], 
                         contour_kwargs_list=[{}, {}],
                         cb_kwargs=dict(bbox_to_anchor=(1.2, 1.1), loc='upper right'), 
                         **kwargs):
    # draw contours
    if contour_name is not None:
        qs = contour_qs
        pi0_colors = contour_colors
        pi0_ls = contour_ls
        skymap_plotter_insert.plot_template_contour(contour_name, quantiles=qs, color=pi0_colors, ls=pi0_ls, 
                                                    smearing_deg=smearing_deg, kwargs_list=contour_kwargs_list)
    
        # dummy legend
        for color, ls, kwargs_i, q in zip(pi0_colors, pi0_ls, contour_kwargs_list, qs):
            if contour_name == 'pi0':
                label = '$\pi^0_\mathrm{s}$' + ' {:1.0f}%'.format(q*100)
            elif contour_name == 'kra5':
                label = 'KRA-$\gamma^5_\mathrm{s}$' + ' {:1.0f}%'.format(q*100)
            elif contour_name == 'kra50':
                label = 'KRA-$\gamma^{50}_\mathrm{s}$' + ' {:1.0f}%'.format(q*100)
            else:
                raise ValueError
            ax_insert.plot(np.nan, np.nan, color=color, ls=ls, label=label, **kwargs_i)
        ax_insert.legend(**cb_kwargs)
    
    if adjust_cb:
        vmin, vmax = skymap_plotter_insert.pc.get_clim()
        if 'n_cb_ticks' in kwargs:
            ticks = np.linspace(vmin, vmax, kwargs['n_cb_ticks'])
        else:
            ticks = np.linspace(vmin, vmax, 9)

        skymap_plotter_insert.cb.set_ticks(ticks)
    
    # set limits and ticks
    if modify_yticks:
        ymin = -np.deg2rad(latitude_deg)
        ymax = np.deg2rad(latitude_deg)
        ax_insert.set_ylim(ymin, ymax)
        ax_insert.set_yticks([ymin,0,ymax])
        #ax_insert.set_yticklabels(['{:1.0f}°'.format(x) for x in [-latitude_deg, 0, latitude_deg]])
        ax_insert.set_yticklabels(['{:1.0f}°'.format(-latitude_deg), '$b = 0°$', '{:1.0f}°'.format(latitude_deg)])
    if modify_xticks:
        xticks_deg = [-180,-120, -60, 0, 60, 120, 180]
        ax_insert.set_xticks(np.deg2rad(xticks_deg))
        xticklabels = []
        for x in xticks_deg[::-1]:
            if x == 0.:
                xticklabels.append('$l = {:1.0f}°$'.format(x))
            else:
                xticklabels.append('{:1.0f}°'.format(x))
        ax_insert.set_xticklabels(xticklabels)
    #ax_insert.set_xlabel('Galactic Longitude $l$ / °')
    #ax_insert.set_ylabel('Galactic Latitude $b$ / °')

def draw_skymap_insert(
            skymap, fig, ax_insert, label, smearing_deg=7, latitude_deg=15, 
            cb_kw={'pad': 0.3}, cb_kw_direct={}, insert_kwargs={}, **kwargs,
        ):
    skymap_plotter_insert = SkymapPlotter(
        skymap=skymap, fig=fig, ax=ax_insert, coord='G', label=label,
        cb_kw=cb_kw, annotate=False,
        cb_kw_direct=cb_kw_direct,
        **kwargs
    )
    _insert_kwargs = deepcopy(kwargs)
    _insert_kwargs.update(insert_kwargs)
    adjust_skymap_insert(
        skymap_plotter_insert=skymap_plotter_insert, 
        ax_insert=ax_insert, 
        smearing_deg=smearing_deg, 
        latitude_deg=latitude_deg,
        **_insert_kwargs
    )
    # turn grid off
    ax_insert.grid(False)
    
    return skymap_plotter_insert

def draw_gp_box(skymap_plotter, ax, latitude_deg, color='0.7', marker='.', s=0.005, n_points=5000):
    # draw box
    l = np.linspace(-np.pi, np.pi, n_points)
    theta_b_upper = np.pi/2 + np.deg2rad(latitude_deg) * np.ones_like(l)
    theta_b_lower = np.pi/2 - np.deg2rad(latitude_deg) * np.ones_like(l)
    
    for theta_b in [theta_b_lower, theta_b_upper]:
        box_theta, box_phi = skymap_plotter.galactic_to_equatorial_coords(theta=theta_b, phi=l)
        x, y = skymap_plotter.convert_theta_phi_to_mpl_coords(theta=box_theta, phi=box_phi)
        ax.scatter(x, y, color=color, marker=marker, s=s)

def draw_connected_box(skymap_plotter, skymap_plotter_insert, lon_range_deg=[-50, 30], 
                       lat_range_deg=[-15, 15],
                       color=(0.984, 0.012, 0.922, 1.0), color_connector=(0.984, 0.012, 0.922, 1.0),
                       marker='.', s=0.005, ls='--', ls_connector=':', lw=2.,
                       n_points=5000,
                       add_connecting_lines=True,
                      ):
    
    lon_range = np.deg2rad(lon_range_deg)
    lat_range = np.deg2rad(lat_range_deg)
    
    # draw box in equatorial map
    lon_list = [
        np.linspace(*lon_range, n_points), 
        np.linspace(*lon_range, n_points), 
        np.ones(n_points) * lon_range[0],
        np.ones(n_points) * lon_range[1],
    ]
    lat_list = [
        np.ones(n_points) * lat_range[0],
        np.ones(n_points) * lat_range[1],
        np.linspace(*lat_range, n_points), 
        np.linspace(*lat_range, n_points), 
    ]
    for lon, lat in zip(lon_list, lat_list):
        phi = lon[::-1]
        theta_b = np.pi/2. + lat
        box_theta, box_phi = skymap_plotter.galactic_to_equatorial_coords(theta=theta_b, phi=phi, rot=0.)
        x, y = skymap_plotter.convert_theta_phi_to_mpl_coords(theta=box_theta, phi=box_phi)
        #skymap_plotter.ax.scatter(x, y, color=color, marker=marker, s=s)
        skymap_plotter.ax.plot(x, y, color=color, ls=ls)
    
    # draw box in galactic map
    for lon, lat in zip(lon_list, lat_list):
        lon_r = -lon
        skymap_plotter_insert.ax.plot(lon_r, lat*0.98, ls=ls, color=color, lw=lw)
        
    if add_connecting_lines:
        ax = skymap_plotter.ax
        ax_gal = skymap_plotter_insert.ax
        
        # compute x, y coords in equatorial map
        phi = np.array([lon_range[0], lon_range[1], lon_range[0], lon_range[1]])
        theta_b = np.pi/2. + np.array([lat_range[0], lat_range[0], lat_range[1], lat_range[1]])
        box_theta, box_phi = skymap_plotter.galactic_to_equatorial_coords(theta=theta_b, phi=phi, rot=0.)
        x, y = skymap_plotter.convert_theta_phi_to_mpl_coords(theta=box_theta, phi=box_phi)
        
        # compute x, y coords in galactic map
        x_gal = -np.array([lon_range[0], lon_range[1], lon_range[0], lon_range[1]])
        y_gal = np.array([lat_range[1], lat_range[1], lat_range[0], lat_range[0]])
        
        for i in range(len(x)):
            con = ConnectionPatch(xyA=(x[i], y[i]), xyB=(x_gal[i], y_gal[i]), 
                                  coordsA="data", coordsB="data", ls=ls_connector,
                                  axesA=ax, axesB=ax_gal, color=color_connector)
            ax.add_artist(con)
            
            if i < 2:
                con_gal = ConnectionPatch(xyA=(x_gal[i], y_gal[i]), xyB=(x[i], y[i]), 
                                      coordsA="data", coordsB="data", ls=ls_connector,
                                      axesA=ax_gal, axesB=ax, color=color_connector)
                ax_gal.add_artist(con_gal)
        

def make_skymap_insert_plot(skymap, label=None, latitude_deg=15, smearing_deg=7, fig=None, ax=None, 
                            ax_insert=None, box_style='total', color_box='0.7', color_coord='0.3', 
                            insert_kwargs={},
                            **kwargs):
    
    # define figure and axes
    if fig is None:
        fig = plt.figure(constrained_layout=True, figsize=(9, 6))
        gs = fig.add_gridspec(2, 3, height_ratios=[3, 1], width_ratios=[1, 9, 1])
        ax = fig.add_subplot(gs[0, :], projection='aitoff')
        ax_insert = fig.add_subplot(gs[1, 1])
    
    # draw equatorial coordinates
    skymap_plotter = SkymapPlotter(skymap=skymap, fig=fig, ax=ax, **kwargs)
    skymap_plotter.cb.remove() 
    
    # Draw Insert
    skymap_plotter_insert = draw_skymap_insert(
        skymap=skymap, fig=fig, ax_insert=ax_insert, label=label, 
        latitude_deg=latitude_deg, smearing_deg=smearing_deg, insert_kwargs=insert_kwargs, **kwargs)
    
    # draw box
    if box_style == 'fraction':
        
        # draw box
        draw_gp_box(skymap_plotter=skymap_plotter, ax=ax, latitude_deg=latitude_deg)
        
        # draw connected box
        draw_connected_box(
            skymap_plotter=skymap_plotter, skymap_plotter_insert=skymap_plotter_insert,
            add_connecting_lines=True,
            color=color_box,
            color_connector='0.7', ls_connector=':',
            #lon_range_deg=[-42, 25], lat_range_deg=[-10, 10],
        )
        
    elif box_style == 'total':
                
        # draw box
        draw_gp_box(skymap_plotter=skymap_plotter, ax=ax, latitude_deg=latitude_deg, color=color_box, n_points=5000)
        
        # plain box
        latitude = np.deg2rad(latitude_deg)
        lw_box = 4
        ax_insert.plot((np.pi, -np.pi), (-latitude, -latitude), color=color_box, lw=lw_box)
        ax_insert.plot((np.pi, -np.pi), (latitude, latitude), color=color_box, lw=lw_box)
        ax_insert.plot((np.pi, np.pi), (latitude, -latitude), color=color_box, lw=lw_box)
        ax_insert.plot((-np.pi, -np.pi), (latitude, -latitude), color=color_box, lw=lw_box)
        
    else:
        raise ValueError('Unknown box style: {}'.format(box_style))
    
    # add coordinate labels
    ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color=color_coord, transform=ax.transAxes)
    ax_insert.text(0.9, 0.12,'Galactic Coord.', ha='right', va='bottom', color=color_coord, transform=ax_insert.transAxes)
        
    return fig, skymap_plotter, skymap_plotter_insert

#### Combined skymap with insert: nsigma (Paper Plot)

In [ ]:
# -------------------------------------
# Compute skymap nsigma with boundaries
# -------------------------------------
ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])

min_val = -0.52
mask = ss_nsigmas <= min_val
ss_nsigmas[mask] = min_val

mask_pole = ss_trial[3] == 0.
ss_nsigmas[mask_pole] = np.nan

# -------------
# norm and cmap
# -------------
cmap = plt.cm.RdBu_r  # define the colormap
N_colors = cmap.N // 2
print('N_colors', N_colors)
cmaplist = [cmap(i) for i in range(N_colors, cmap.N)]
cmaplist[0] = (1., 1., 1., 1.) # force first bin to be completely white

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0, 4, 9)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

color_box = '0.7'
#color_box = np.array([64, 73, 255]) / 255.
fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
    skymap=ss_nsigmas, cmap=cmap, norm=norm, label='Pre-trial significance / $\sigma$', color_box=color_box)
#skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
#skymap_plotter_insert.plot_hotspots(marker='x', color='0.8', s=20)

# add axis and panel labels
if True:
    skymap_plotter.ax.text(
        -0.07, 0.5,r'Declination $\delta$', 
        rotation=90.,
        ha='right', va='center', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    skymap_plotter.ax.text(
        0.5, -0.06, r'Right Ascension $\alpha$', 
        ha='center', va='bottom', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    
    skymap_plotter.ax.text(
        0.0, 1.01,'A', 
        fontsize=18,
        ha='left', va='top', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    skymap_plotter.ax.text(
        0.0, -0.03,'B', 
        fontsize=18,
        ha='left', va='top', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    

# -----------
# add Catalog
# -----------
#cats_to_plot = ['Source List']
cats_to_plot = ['pwn', 'snr', 'unid']
#cats_to_plot = ['pwn', 'snr', 'unid', 'Source List']
for cat_str, color, marker, s in zip(
            ['pwn', 'snr', 'unid', 'Source List'], 
            [(0.19, 0.87, 0.768), '0.8', (0.90, 0.92, 0.01), '1.0'], 
            ['*', '^', '.', 'P'], 
            [70, 50, 70, 20],
        ):
    if cat_str not in cats_to_plot: continue
    if cat_str != 'Source List':
        label = cat_str.upper()
    else:
        label = cat_str
    skymap_plotter.plot_catalog(marker=marker, color=color, keys=[cat_str], edgecolor='0.0', linewidth=0.4, s=s, labels=[label])
    skymap_plotter_insert.plot_catalog(marker=marker, color=color, keys=[cat_str], edgecolor='0.0', linewidth=0.4, s=s, labels=[label])

# -----------------------------
# add contour for Fermi-Bubbles
# -----------------------------
color_bubbles = [(0.19, 0.87, 0.768), (0.19, 0.87, 0.768)]
skymap_plotter.plot_template_contour('fermibubbles_50TeV', quantiles=[0.2, 0.5], color=color_bubbles, ls=['--', '-.'], smearing_deg=7)
skymap_plotter_insert.plot_template_contour('fermibubbles_50TeV', quantiles=[0.2, 0.5], color=color_bubbles, ls=['--', '-.'], smearing_deg=7)
skymap_plotter_insert.ax.plot(np.inf, np.inf, color=color_bubbles[0], ls='--', label='$\mathrm{FB}_s$ 20%')
skymap_plotter_insert.ax.plot(np.inf, np.inf, color=color_bubbles[0], ls='-.', label='$\mathrm{FB}_s$ 50%')
# -----------------------------

#skymap_plotter.ax.legend(bbox_to_anchor=(0.98, 0.15), loc='upper right')
skymap_plotter_insert.ax.legend(bbox_to_anchor=(1.24, -0.1), loc='lower right') # 1.2, -0.1

fig.savefig('{}/skymap_nsigma_combined.png'.format(plot_dir), dpi=300, bbox_inches='tight')
fig.savefig('{}/skymap_nsigma_combined.pdf'.format(plot_dir), dpi=300, bbox_inches='tight')

skymap_plotter_insert.ax.text(
    -.2, 1.01, embargo_str, 
    ha='left', va='bottom', color='red', fontsize=18,
    transform=skymap_plotter_insert.ax.transAxes,
)
fig.savefig('{}/skymap_nsigma_combined__embargo.png'.format(plot_dir), dpi=300, bbox_inches='tight')
fig.savefig('{}/skymap_nsigma_combined__embargo.pdf'.format(plot_dir), dpi=300, bbox_inches='tight')


#### Combined skymap with insert: gamma (Paper Plot)

In [ ]:
def make_2d_gamma_insert_plot(smearing_deg=7, latitude_deg=15, box_style='total', 
                              color_box=(0.984, 0.012, 0.922, 0.3), color_coord='0.3',
                              ss_trial=ss_trial,
                              **kwargs):
    # Define figure layout and axes
    fig = plt.figure(constrained_layout=True, figsize=(9, 6))
    # we use a 5x5 grid since usage of hspace seems to mess up layout
    gs = fig.add_gridspec(5, 5, height_ratios=[4, 0.3, 1, 0.1, 0.3], width_ratios=[1, 2, 5, 2, 1], hspace=0)
    ax = fig.add_subplot(gs[0, :], projection='aitoff')
    ax_insert = fig.add_subplot(gs[2, 1:4])
    ax_cmatrix = fig.add_subplot(gs[4, 2])

    # plot 2D gamma skymap in Equatorial
    skymap_plotter = draw_2d_gamma(fig=fig, ax=ax, coord='C', ss_trial=ss_trial)
    skymap_plotter.cb.remove() 
    
    # plot 2D gamma skymap insert in Galactic
    skymap_plotter_insert = draw_2d_gamma(fig=fig, ax=ax_insert, coord='G', annotate=False, ss_trial=ss_trial)
    skymap_plotter_insert.cb.remove()
    adjust_skymap_insert(
        skymap_plotter_insert=skymap_plotter_insert, 
        ax_insert=ax_insert, 
        smearing_deg=smearing_deg, 
        latitude_deg=latitude_deg,
        adjust_cb=False,
        cb_kwargs=dict(bbox_to_anchor=(1.19, 1.1), loc='upper right'),
        **kwargs
    )
    
    # draw box
    if box_style == 'fraction':
        
        # draw box
        draw_gp_box(skymap_plotter=skymap_plotter, ax=ax, latitude_deg=latitude_deg)
        
        # draw connected box
        draw_connected_box(
            skymap_plotter=skymap_plotter, skymap_plotter_insert=skymap_plotter_insert,
            add_connecting_lines=True,
            color_connector='0.7', ls_connector=':',
            #lon_range_deg=[-42, 25], lat_range_deg=[-10, 10],
        )
        
    elif box_style == 'total':
                
        # draw box
        draw_gp_box(skymap_plotter=skymap_plotter, ax=ax, latitude_deg=latitude_deg, color=color_box, n_points=5000)
        
        # plain box
        latitude = np.deg2rad(latitude_deg)
        lw_box = 4
        ax_insert.plot((np.pi, -np.pi), (-latitude, -latitude), color=color_box, lw=lw_box)
        ax_insert.plot((np.pi, -np.pi), (latitude, latitude), color=color_box, lw=lw_box)
        ax_insert.plot((np.pi, np.pi), (latitude, -latitude), color=color_box, lw=lw_box)
        ax_insert.plot((-np.pi, -np.pi), (latitude, -latitude), color=color_box, lw=lw_box)
        
    else:
        raise ValueError('Unknown box style: {}'.format(box_style))
        
    
    # add grid to insert
    #ax_insert.grid(color='.5', alpha=.5)

    # plot cbar matrix
    plot_2d_colorbar(ax_cmatrix=ax_cmatrix, n_gamma_bins=n_gamma_bins)

    # redraw grid and annotations
    skymap_plotter.annotate_skymap(ax=skymap_plotter.ax, sp=skymap_plotter.sp)
    
    # add coordinate labels
    ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color=color_coord, transform=ax.transAxes)
    ax_insert.text(0.9, 0.12,'Galactic Coord.', ha='right', va='bottom', color=color_coord, transform=ax_insert.transAxes)
    
    return fig, skymap_plotter, skymap_plotter_insert



fig, skymap_plotter, skymap_plotter_insert = make_2d_gamma_insert_plot()

# add axis and panel labels
if True:
    skymap_plotter.ax.text(
        -0.07, 0.5,r'Declination $\delta$', 
        rotation=90.,
        ha='right', va='center', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    skymap_plotter.ax.text(
        0.5, -0.06, r'Right Ascension $\alpha$', 
        ha='center', va='bottom', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    
    skymap_plotter.ax.text(
        -0.03, 1.01,'A', 
        fontsize=18,
        ha='left', va='top', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    skymap_plotter.ax.text(
        -0.03, -0.01,'B', 
        fontsize=18,
        ha='left', va='top', color='0.', 
        transform=skymap_plotter.ax.transAxes,
    )
    
    
#skymap_plotter.plot_catalog()
#skymap_plotter.plot_hotspots()
#skymap_plotter_insert.plot_hotspots()
#skymap_plotter.plot_fermi_sources()
fig.savefig('{}/skymap_gamma_combined.png'.format(plot_dir), bbox_inches="tight", dpi=300)
fig.savefig('{}/skymap_gamma_combined.pdf'.format(plot_dir), bbox_inches="tight", dpi=300)

skymap_plotter_insert.ax.text(
    -.2, 1.01, embargo_str, 
    ha='left', va='bottom', color='red', fontsize=18,
    transform=skymap_plotter_insert.ax.transAxes,
)
fig.savefig('{}/skymap_gamma_combined__embargo.png'.format(plot_dir), bbox_inches="tight", dpi=300)
fig.savefig('{}/skymap_gamma_combined__embargo.pdf'.format(plot_dir), bbox_inches="tight", dpi=300)

#### Compute template area

In [ ]:
for i, template_str in enumerate(['pi0', 'kra5', 'kra50']):
    for j, smearing_deg in enumerate([0, 7, 15]):
        print('Template {} with smearing of: {}°'.format(template_str, smearing_deg))
        template = get_smeared_template(template_str, smearing=smearing_deg)
        
        nside = hp.npix2nside(len(template))
        pix_area = hp.nside2pixarea(nside)
        pix_area_deg = hp.nside2pixarea(nside, degrees=True)
        
        # convert template to contain the probability
        # for each pixel, e.g. sum over pixel probs is 1
        template_prob = template * pix_area
        
        # sort these and compute cumulative values in order
        # to find the threshold value
        sorted_probs = np.sort(template_prob)
        cum_probs = np.cumsum(sorted_probs)
        for q in [0.2, 0.5]:
            idx_threshold = np.searchsorted(cum_probs, 1. - q)
            prob_threshold = sorted_probs[idx_threshold]
            mask = template_prob >= prob_threshold
            area = np.sum(mask) * pix_area
            area_deg = np.sum(mask) * pix_area_deg
            print('  {:3.0f}%: {:3.3f} sr | {:3.3f} sq. deg. | {:4.1f}% of sky'.format(q*100., area, area_deg, area/(4*np.pi) *100.))
        #np.sum(template_prob)

#### Templates in combined skymap <s>(Paper Plot)</s> [OLD - DEPRECATED]

In [ ]:

# each combined skymap needs:
# add_gridspec(2, 3, height_ratios=[3, 1], width_ratios=[1, 9, 1])
# for 3 x 2, we'll need 6, 6
n_col = 2
n_row = 3

fig = plt.figure(constrained_layout=True, figsize=(9 * n_col, 6 * n_row))
gs = fig.add_gridspec(2*n_row, 3*n_col, height_ratios=[3, 1] * n_row, width_ratios=[1, 9, 1] * n_col)

# create axes
ax_matrix = []
ax_insert_matrix = []
for i in range(n_col):
    ax_rows = []
    ax_insert_rows = []
    for j in range(n_row):
        ax_rows.append(fig.add_subplot(gs[2*j, 3*i:3*(i+1)], projection='aitoff'))
        ax_insert_rows.append(fig.add_subplot(gs[2*j + 1, 3*i+1]))
    ax_matrix.append(ax_rows)
    ax_insert_matrix.append(ax_insert_rows)

# ---------------
# define colormap
# ---------------
cmap = plt.cm.viridis  
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0, 1, 18)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# ---------------

# define labels
label_name_dict = {
    'pi0': r'$\pi^0$',
    'kra5': r'KRA-$\gamma$',
}

# draw templates
for i, template_str in enumerate(['pi0', 'kra5']):
    for j, smearing_deg in enumerate([0, 7, 15]):
        if smearing_deg == 0:
            label = r'P({}) without smearing'.format(label_name_dict[template_str])
        else:
            label = r'P({}) with {:1.0f}° smearing'.format(label_name_dict[template_str], smearing_deg)
        _, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
            fig=fig, ax=ax_matrix[i][j], ax_insert=ax_insert_matrix[i][j],
            skymap=get_smeared_template(template_str, smearing=smearing_deg),
            #label=r'$-\log_{10}(p_\mathrm{local})$',
            label=label,
            #vmin=0, vmax=1., 
            cmap=cmap, norm=norm,
            gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=5, gp_lw=1.,
            insert_kwargs=dict(contour_name=None),
            color_coord='1.0',
            color_box='1.0',
        )
        
        # make individiual plot
        if True:
            fig_i, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
                skymap=get_smeared_template(template_str, smearing=smearing_deg),
                label=label,
                cmap=cmap, norm=norm,
                gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=5, gp_lw=1.,
                insert_kwargs=dict(contour_name=None),
                color_coord='1.0',
                color_box='1.0',
            )
            fig_i.savefig('{}/template_{}_{}deg.png'.format(plot_dir, template_str, smearing_deg))   
            

fig.savefig('{}/template_overview.png'.format(plot_dir), dpi=300)   
fig.savefig('{}/template_overview.pdf'.format(plot_dir), dpi=300)  

ax_insert_matrix[1][-1].text(
    -.7, 1.01, embargo_str, 
    ha='left', va='bottom', color='red', fontsize=42,
    transform=ax_insert_matrix[1][-1].transAxes,
)
fig.savefig('{}/template_overview__embargo.png'.format(plot_dir), dpi=300)    
fig.savefig('{}/template_overview__embargo.pdf'.format(plot_dir), dpi=300)  


#### Templates in combined skymap (Paper Plot)

In [ ]:
import matplotlib.patheffects as pe

fig = plt.figure(constrained_layout=True, figsize=(9, 9))
gs = fig.add_gridspec(9, 3, height_ratios=[10] * 3 + [3] + [10] * 3 + [1, 2], width_ratios=[1, 3, 1])

# create axes
axes = [fig.add_subplot(gs[0, :])]
for i in range(1, 3):
    axes.append(fig.add_subplot(gs[i, :], sharex=axes[0]))
    
axes.append(fig.add_subplot(gs[4, :]))
for i in range(5, 7):
    axes.append(fig.add_subplot(gs[i, :], sharex=axes[3]))
    
ax_cb = fig.add_subplot(gs[8, 1])

# ---------------
# define colormap
# ---------------
cmap = plt.cm.viridis  
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0., 1.2, 18)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# ---------------

# define labels
label_name_dict = {
    'pi0': r'$\pi^0$',
    'kra5': r'KRA$_\gamma^{5}$',
    'kra50': r'KRA$_\gamma^{50}$',
}

# draw templates
counter = 0
for i, template_str in enumerate(['pi0', 'kra5']):
    for j, smearing_deg in enumerate([0, 7, 15]):
        if smearing_deg == 0:
            label = r'{}, no smearing'.format(label_name_dict[template_str])
        else:
            label = r'{}, {:1.0f}° smearing'.format(label_name_dict[template_str], smearing_deg)
            
        skymap_plotter_insert = draw_skymap_insert(
            skymap=get_smeared_template(template_str, smearing=smearing_deg),
            fig=fig,
            ax_insert=axes[counter],
            label=r'Density / sr$^{-1}$',
            smearing_deg=smearing_deg,
            latitude_deg=30,
            cmap=cmap, norm=norm,
            gp_kw=dict(color='0.3', alpha=0.5), 
            n_cb_ticks=5, gp_lw=1.,
            insert_kwargs=dict(contour_name=None, modify_xticks=False, modify_yticks=False),
            cb_kw_direct={'pad': 0.3, 'cax': ax_cb},
            #color_coord='1.0',
            #color_box='1.0',
        )
        
        axes[counter].text(
            .003, 0.0, label, 
            ha='left', va='bottom', color='1.0', fontsize=14,
            transform=axes[counter].transAxes,
            path_effects=[pe.Stroke(linewidth=2, foreground='0.'), pe.Normal()],
        )
        
        # increment axis counter
        counter += 1

axes[0].set_title(r'$\pi^0$ Template')
axes[3].set_title(r'KRA$_\gamma^{5}$ Template')

# modify y ticks
yticks_deg = np.r_[-30:30.1:15]
for ax in axes:
    ax.set_yticks(np.deg2rad(yticks_deg))
    
    yticklabels = []
    for y in yticks_deg[::-1]:
        if y == 0.:
            yticklabels.append('$b = {:1.0f}°$'.format(y))
        else:
            yticklabels.append('{:1.0f}°'.format(y))
    
    ax.set_yticklabels(yticklabels)
    ax.set_ylim(min(np.deg2rad(yticks_deg)), max(np.deg2rad(yticks_deg)))
    
# modify x ticks
xticks_deg = np.r_[-180:180.1:60]
for i, ax in enumerate(axes):
    ax.set_xticks(np.deg2rad(xticks_deg))
    xticklabels = []
    for x in xticks_deg[::-1]:
        if x == 0.:
            xticklabels.append('$l = {:1.0f}°$'.format(x))
        else:
            xticklabels.append('{:1.0f}°'.format(x))
    ax.set_xticklabels(xticklabels)

panel_labels = ['A', 'B', 'C', 'D', 'E', 'F']
for i, ax in enumerate(axes):
    if i not in [2, 5]:
        plt.setp(ax.get_xticklabels(), visible=False)
    
    # set panel labels
    ax.text(
        .003, 0.95, panel_labels[i], 
        ha='left', va='top', color='1.0', fontsize=18,
        transform=ax.transAxes,
        path_effects=[pe.Stroke(linewidth=2, foreground='0.'), pe.Normal()],
    )

fig.savefig('{}/template_overview_galactic.png'.format(plot_dir), bbox_inches='tight', dpi=300)   
fig.savefig('{}/template_overview_galactic.pdf'.format(plot_dir), bbox_inches='tight', dpi=300)  


#### Templates in combined map over isotropic expectation

In [ ]:

# each combined skymap needs:
# add_gridspec(2, 3, height_ratios=[3, 1], width_ratios=[1, 9, 1])
# for 3 x 2, we'll need 6, 6
n_col = 2
n_row = 3

fig = plt.figure(constrained_layout=True, figsize=(9 * n_col, 6 * n_row))
gs = fig.add_gridspec(2*n_row, 3*n_col, height_ratios=[3, 1] * n_row, width_ratios=[1, 9, 1] * n_col)

# create axes
ax_matrix = []
ax_insert_matrix = []
for i in range(n_col):
    ax_rows = []
    ax_insert_rows = []
    for j in range(n_row):
        ax_rows.append(fig.add_subplot(gs[2*j, 3*i:3*(i+1)], projection='aitoff'))
        ax_insert_rows.append(fig.add_subplot(gs[2*j + 1, 3*i+1]))
    ax_matrix.append(ax_rows)
    ax_insert_matrix.append(ax_insert_rows)

# ---------------
# define colormap
# ---------------
cmap = plt.cm.RdBu_r 
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(1e-3, 1e3, 18)
bounds = np.linspace(-3, 3, 18) + 1.
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# ---------------

# define labels
label_name_dict = {
    'pi0': r'$\pi^0$',
    'kra5': r'KRA-$\gamma$',
}

# draw templates
for i, template_str in enumerate(['pi0', 'kra5']):
    for j, smearing_deg in enumerate([3, 7, 15]):
        skymap = get_smeared_template(template_str, smearing=smearing_deg)
        
        nside = float(hp.get_nside(skymap))
        skymap_ratio = skymap / (hp.nside2pixarea(nside) * nside)
        _, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
            fig=fig, ax=ax_matrix[i][j], ax_insert=ax_insert_matrix[i][j],
            skymap=np.log10(skymap_ratio),
            #label=r'$-\log_{10}(p_\mathrm{local})$',
            label=r'$\log_{10}($' + r'P({})'.format(label_name_dict[template_str]) + r'/ $P_\mathrm{iso}$' + ' with {:1.0f}° smearing'.format(smearing_deg),
            #vmin=0, vmax=1., 
            cmap=cmap, norm=norm,
            gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=5, gp_lw=1.,
            insert_kwargs=dict(contour_name=None),
            color_coord='1.0',
            color_box='1.0',
        )

fig.savefig('{}/template_overview_over_isotropic.png'.format(plot_dir))   


In [ ]:
hp.nside2pixarea(nside) * nside

#### Combined skymap with insert: pvalue

In [ ]:
fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
    skymap=ss_trial[0],
    label=r'$-\log_{10}(p_\mathrm{local})$',
    vmin=0, vmax=6, 
    gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=2, gp_lw=1.
)
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
skymap_plotter_insert.plot_hotspots(marker='x', color='0.8', s=20)
fig.savefig('{}/skymap_pvalues_combined.png'.format(plot_dir))



#### Skymap in Galactic Coordinates: nsigmas

In [ ]:
from scipy import stats

# -------------------------------------
# Compute skymap nsigma with boundaries
# -------------------------------------
ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])

min_val = -0.52
mask = ss_nsigmas <= min_val
ss_nsigmas[mask] = min_val

mask_pole = ss_trial[3] == 0.
ss_nsigmas[mask_pole] = np.nan

# -------------
# norm and cmap
# -------------
cmap = plt.cm.RdBu_r  # define the colormap
N_colors = cmap.N // 2
print('N_colors', N_colors)
cmaplist = [cmap(i) for i in range(N_colors, cmap.N)]
cmaplist[0] = (1., 1., 1., 1.) # force first bin to be completely white

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0, 4, 9)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

skymap_plotter = SkymapPlotter(
    skymap=ss_nsigmas, cmap=cmap, norm=norm, label=r'Significance: $n\cdot \sigma$', 
    coord='G', gp_kw=dict(color='0.3', alpha=0.5), gp_lw=1., n_cb_ticks=9)
#skymap_plotter.plot_catalog(marker='x', color='k')
#skymap_plotter.plot_hotspots(marker='x', color='0.8')
#skymap_plotter.plot_fermi_sources(marker='+', color='green')

smearing_deg = 13
q = 0.5
#skymap_plotter.plot_template_contour('pi0', quantiles=[q], color=['0.2'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('kra5', quantiles=[q], color=['green'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('pi0', quantiles=[q, 0.5], color=['0.2', '0.5'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('kra5', quantiles=[q, 0.5], color=['green', 'green'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour_points('pi0', level=q, color='0.2', smearing_deg=smearing_deg, delta=0.004)
#skymap_plotter.plot_template_contour_points('kra5', level=q, color='green', smearing_deg=smearing_deg, delta=0.004)
skymap_plotter.draw_equator()

skymap_plotter.ax.text(0.84, 0.16,'Galactic Coord.', ha='right', va='bottom', color='0.3', transform=skymap_plotter.ax.transAxes)

skymap_plotter.fig.savefig('{}/skymap_nsigma_galactic.png'.format(plot_dir))
skymap_plotter.fig.savefig('{}/skymap_nsigma_galactic.pdf'.format(plot_dir))
#skymap_plotter.fig.savefig('{}/skymap_nsigma_galactic_s{:1.1f}_q{:1.1f}.png'.format(plot_dir, smearing_deg, q))


#### Skymap in Galactic Coordinates: p-value

In [ ]:
skymap_plotter = SkymapPlotter(
    skymap=ss_trial[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, gp_kw=dict(color='0.3', alpha=0.5), gp_lw=1., coord='G')
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
skymap_plotter.draw_equator(color='0.6', s=0.005)
skymap_plotter.fig.savefig('{}/skymap_pvalues_galactic.png'.format(plot_dir))


#### Skymap in Galactic Coordinates: gamma

In [ ]:
cmap = plt.cm.viridis  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(1, 4, 17)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

gamma_masked = np.array(ss_trial[3])
gamma_masked[10**(-ss_trial[0]) > 0.5] = np.nan


skymap_plotter = SkymapPlotter(skymap=gamma_masked, cmap=cmap, norm=norm, label=r'$\gamma$', coord='G')
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8')
#skymap_plotter.plot_fermi_sources(marker='+', color='green')
#skymap_plotter.plot_template_contour('pi0', level=0.5, color='0.2', smearing_deg=5)
#skymap_plotter.plot_template_contour('kra5', level=0.5, color='green', smearing_deg=5)
skymap_plotter.draw_equator()

skymap_plotter.fig.savefig('{}/skymap_gamma_galactic.png'.format(plot_dir))

#### Templates in Galactic

In [ ]:
for template_str in ['pi0', 'kra5']:
    for smearing in [0., 5, 10]:
        for coord in ['G', 'C']:
            template = get_smeared_template(template_str, smearing=smearing)
            skymap_plotter = SkymapPlotter(
                skymap=template, cmap='viridis', 
                label=r'P({}) with {:1.0f}° smearing'.format(template_str, smearing), 
                coord=coord,
            )
            skymap_plotter.draw_equator()
            skymap_plotter.fig.savefig('{}/template_{}_smearing_{:1.1f}_{}.png'.format(
                plot_dir, template_str, smearing, coord))        

## Contribution Map

#### Helper Methods

In [ ]:
from multiprocessing import Pool
from tqdm.notebook import tqdm_notebook as tqdm
from dnn_cascade_selection.utils.notebook import ps_pdf

def get_energy_and_space_contribution(trial, tr, gamma=2.7):
    L = tr.get_one_llh_from_trial(trial)
    
    res = L.fit(**tr.fitter_args)
    ns = res[1]['ns']
    N = float(len(trial.evss[0][0]))
    print(res, ns, N)
    
    space_eval = cy.inspect.get_space_eval(L, -1, 0) # 0: background events (1 would be for signal events)
    energy_eval = cy.inspect.get_energy_eval(L, -1, 0)
    StoB_space_ss = space_eval(gamma=gamma)[1] 
    SoB_energy = energy_eval(gamma=gamma)[0]
    SoB_space = space_eval(gamma=gamma)[0] 
    #w = (SoB_space - StoB_space_ss) * SoB_energy
    #lr = w * ns/N + 1.
    space = SoB_space #(SoB_space - StoB_space_ss)
    energy = SoB_energy
    return space, energy, ns/N

def get_lr_from_trial(trial, tr, gamma=2.7):
    """Get event likelihood-ratio value of signal-subtracted likelihood
    
    Info here:
        https://wiki.icecube.wisc.edu/index.php/
        Cascade_7yr_PS_GP/Galactic_Source_Search_Methods#Signal-Subtracted_Likelihood
    """
    L = tr.get_one_llh_from_trial(trial)
    
    res = L.fit(**tr.fitter_args)
    ns = res[1]['ns']
    N = float(len(trial.evss[0][0]))
    print(res, ns, N)
    
    space_eval = cy.inspect.get_space_eval(L, -1, 0) # 0: background events (1 would be for signal events)
    energy_eval = cy.inspect.get_energy_eval(L, -1, 0)
    StoB_space_ss = space_eval(gamma=gamma)[1] 
    SoB_energy = energy_eval(gamma=gamma)[0]
    SoB_space = space_eval(gamma=gamma)[0] 
    w = (SoB_space - StoB_space_ss) * SoB_energy
    lr = w * ns/N + 1.
    return lr
    
def get_lr(template_str, gamma=2.7, seed=None, TRUTH=True):
    """Get event likelihood-ratio value of signal-subtracted likelihood
    
    Info here:
        https://wiki.icecube.wisc.edu/index.php/
        Cascade_7yr_PS_GP/Galactic_Source_Search_Methods#Signal-Subtracted_Likelihood
    """
    trial = tr_dict[template_str].get_one_trial(seed=seed, TRUTH=TRUTH)
    return get_lr_from_trial(trial=trial, tr=tr_dict[template_str], gamma=gamma)

def get_contribution_list(template_str, N, w, ws, seed=None, TRUTH=True):
    trial = tr_dict[template_str].get_one_trial(seed=seed, TRUTH=TRUTH)
    
    ci_list = []
    ra_list = []
    dec_list = []
    sigma_list = []
    for a in tqdm(ws[::-1][:N]):
        mask = trial.evss[0][0].idx == a
        dec = trial.evss[0][0][mask].dec[0]
        ra = trial.evss[0][0][mask].ra[0]
        sigma = trial.evss[0][0][mask].sigma[0]
        w_i = w[a]
        
        #ci_list.append(10**(w_i))
        ci_list.append(w_i)
        ra_list.append(ra)
        dec_list.append(dec)
        sigma_list.append(sigma)
    
    ci_list = np.array(ci_list)
    ra_list = np.array(ra_list)
    dec_list = np.array(dec_list)
    sigma_list = np.array(sigma_list)
    return ci_list, ra_list, dec_list, sigma_list

def compute_pixel(args):
    ipix, nside, ci_list, ra_list, dec_list, sigma_list = args
    theta, phi = hp.pix2ang(nside=nside, ipix=ipix)
    dec = np.pi/2.  - theta
    ra = phi

    ang_dist = cy.coord.delta_angle(
        zenith1=dec,
        azimuth1=ra,
        zenith2=dec_list,
        azimuth2=ra_list,
        latlon=True,
    )
    space_pdf = ps_pdf.von_mises_pdf(ang_dist, sigma=sigma_list)

    pixel_value = np.sum(space_pdf * ci_list)
    return pixel_value
    
def get_contribution_map(template_str, N, w, ws, nside=64, cpus=25, normalize=False, seed=None, TRUTH=True):
    ci_list, ra_list, dec_list, sigma_list = get_contribution_list(
        template_str=template_str, N=N, w=w, ws=ws, seed=seed, TRUTH=TRUTH)
    
    npix = hp.nside2npix(nside)
    arg_list = [(i, nside, ci_list, ra_list, dec_list, sigma_list) for i in range(npix)]
    
    if cpus > 1:
        print('Running pool with {} cpus'.format(cpus))
        

        with Pool(cpus) as p:
            skymap = list(tqdm(p.imap(compute_pixel, arg_list), total=npix))
        skymap = np.array(skymap)
        p.close()
    else:
        skymap = np.zeros(npix)
        for i in tqdm(range(npix), total=npix):
            skymap[i] = compute_pixel(arg_list[i])
    
    # normalize skymap
    if normalize:
        skymap = np.array(skymap) / np.sum(skymap) / hp.nside2pixarea(nside=nside)
    return skymap



#### Space vs Energy Contribution

In [ ]:
space, energy, ns_over_N = get_energy_and_space_contribution(
    trial=tr_dict['pi0'].get_one_trial(TRUTH=True),
    tr=tr_dict['pi0'],
)
ts_space = 2* np.log(space * ns_over_N + 1)
ts_energy = 2* np.log(energy * ns_over_N + 1)
print('energy', np.min(energy), np.max(energy))
print('space', np.min(space), np.max(space))
print('ts_energy', np.min(ts_energy), np.max(ts_energy), np.sum(ts_energy))
print('ts_space', np.min(ts_space), np.max(ts_space), np.sum(ts_space))


In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
bins = np.linspace(-10, 30, 100)
ax.hist(space, bins=bins, histtype='step', label='Space Contribution')
ax.hist(energy, bins=bins, histtype='step', label='Energy Contribution')
ax.legend()
ax.set_yscale('log')


#### Compute event contribution

In [ ]:
lr_dict = {
    'pi0': get_lr('pi0'),
    'kra5': get_lr('kra5', None),
    'kra50': get_lr('kra50', None),
}

ts_dict = {}
for key, lr in lr_dict.items():
    ts = 2 * np.log(lr)
    print(key, np.sum(ts))
    ts_dict[key] = ts



#### Compute top events to get down to 2-sigma

In [ ]:
max_trials = 5000000

top_idx_list = {}
for key, ts in ts_dict.items():
    
    # get bkg dist
    bg_tsd = cy.dists.TSD(bkg_dict[key][:max_trials])
    
    # get ts value at which 2-sigma is achieved
    ts_limit = bg_tsd.isf_nsigma(2.)
    
    # compute how much ts contribution needs to be removed
    ts_delta = np.sum(ts) - ts_limit
    
    print(key, ts_limit, ts_delta, np.sum(ts))
    
    # select top n events until ts of 2-sigma is reached
    sorted_idx = np.argsort(ts)[::-1]
    cum_ts = np.cumsum(ts[sorted_idx])
    idx = np.searchsorted(cum_ts, ts_delta)
    
    remaining_ts_less = np.sum(ts[sorted_idx[idx:]])
    remaining_ts = np.sum(ts[sorted_idx[idx+1:]])
    print('   N events removed: {} | {:3.3f} | {:3.3f} sigma'.format(
        idx, remaining_ts_less, bg_tsd.sf_nsigma(remaining_ts_less)))
    print('   N events removed: {} | {:3.3f} | {:3.3f} sigma'.format(
        idx+1, remaining_ts, bg_tsd.sf_nsigma(remaining_ts)))
    
    top_idx_list[key] = sorted_idx[:idx + 1]
    print(len(top_idx_list[key]))

#### Save top event contributions to file

In [ ]:
for key, idx in top_idx_list.items():

    print(a.bg_data.energy[idx])

    import pandas as pd

    df = pd.DataFrame({
        'run': a.bg_data.run[idx],
        'event': a.bg_data.event[idx],
        'subevent': a.bg_data.subevent[idx],
    })
    df.to_pickle('{}/top_events_{}.pickle'.format(plot_dir, key))


In [ ]:
top_idx_list_ = {}
for key in ['pi0', 'kra5', 'kra50']:
    top_idx_list_[key] = pd.read_pickle('{}/top_events_{}.pickle'.format(plot_dir, key))
top_idx_list_['pi0']

In [ ]:
def get_combined_list(top_idx_list, sort_by='pi0'):
    event_list = {}
    for key, df in top_idx_list.items():
        for i in range(len(df)):
            event = (df['run'].iloc[i], df['event'].iloc[i])
            if event not in event_list:
                event_list[event] = {key: i + 1}
            else:
                event_list[event][key] = i + 1
    print(len(event_list))
    return event_list


get_combined_list(top_idx_list=top_idx_list_)

#### Create contribution maps

In [ ]:
bkg_scrample_seed = 4
lr_bkg_dict = {
    'pi0': get_lr('pi0', seed=bkg_scrample_seed, TRUTH=False),
    'kra5': get_lr('kra5', None, seed=bkg_scrample_seed, TRUTH=False),
    'kra50': get_lr('kra50', None, seed=bkg_scrample_seed, TRUTH=False),
}

ts_bkg_dict = {}
for key, lr in lr_bkg_dict.items():
    ts = 2 * np.log(lr)
    print(key, np.sum(ts))
    ts_bkg_dict[key] = ts
    
    try:
        bg_tsd = cy.dists.TSD(bkg_dict[key])
        total_ts = np.sum(ts)
        print('  {} | ts: {:3.3f} | p-value: {:3.3f} | n-sigma: {:3.3f}'.format(
            key, total_ts, bg_tsd.sf(total_ts), bg_tsd.sf_nsigma(total_ts)))
    except Exception as e:
        print(e)
        pass

print('bkg_scrample_seed', bkg_scrample_seed)

#### Compute contribution of standard candle run

In [ ]:
mask = np.logical_or(a.bg_data.run == 125910, a.bg_data.run == 125911)
for key, ts_vals in ts_dict.items():
    print(key, ts_vals[mask], np.sum(ts_vals[mask]))

#### Compute contribution Maps

In [ ]:
nside = 64

skymaps_all = {}
for key, ts_values in ts_dict.items():
    print('Creating contributions for {}'.format(key))
    
    sorted_idx = np.argsort(ts_values)
    skymaps_all[key] = get_contribution_map(
        template_str=key, N=len(ts_values), w=ts_values, ws=sorted_idx, nside=nside)


In [ ]:
skymaps_all_bkg = {}
for key, ts_values in ts_bkg_dict.items():
    print('Creating contributions for {}'.format(key))
    
    sorted_idx = np.argsort(ts_values)
    skymaps_all_bkg[key] = get_contribution_map(
        template_str=key, N=len(ts_values), w=ts_values, ws=sorted_idx, nside=nside, 
        seed=bkg_scrample_seed, TRUTH=False,
    )
    

#### Save contribution maps to file

In [ ]:
for key, skymap in skymaps_all.items():
    file_name = '{}/contibution_map_{}.csv'.format(plot_dir, key)
    
    # save as csv
    np.savetxt(file_name, skymap, delimiter=',')
    
    print('Done saving {} to {}.'.format(key, file_name))
    
    

In [ ]:
# check how these are saved
if True:
    for key, _ in skymaps_all.items():
        file_name = '{}/contibution_map_{}.csv'.format(plot_dir, key)

        # load from csv
        skymap = np.loadtxt(file_name, delimiter=',')

        hp.mollview(skymap)

#### Make Plots

In [ ]:
for k in skymaps_all.keys():
    print(k, np.min(skymaps_all_bkg[k]), np.max(skymaps_all_bkg[k]))
    print(k, np.min(skymaps_all[k]), np.max(skymaps_all[k]))
    

In [ ]:
import matplotlib.patheffects as pe

# ---------------
# define colormap
# ---------------
cmap = plt.cm.inferno  
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-110, 410, 18)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# ---------------


path_effects = [pe.Stroke(linewidth=3, foreground='0.'), pe.Normal()]
contour_kwargs_list = [dict(path_effects=path_effects), dict(path_effects=path_effects)]

for key in skymaps_all.keys():
    if key == 'pi0':
        label = r'$\pi^0$ TS-contribution / sr'
    elif key == 'kra5':
        label = r'$\mathrm{KRA}_\gamma^5$ TS-contribution / sr'
    elif key == 'kra50':
        label = r'$\mathrm{KRA}_\gamma^{50}$ TS-contribution / sr'
    else:
        raise ValueError()
    
    if True:
        fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
            skymap=skymaps_all[key],
            label=label,
            cmap=cmap, norm=norm,
            insert_kwargs=dict(contour_name=key, contour_colors=['1.0', '0.7'], contour_kwargs_list=contour_kwargs_list),
            gp_kw=dict(color='0.3', alpha=0.5), #n_cb_ticks=2, gp_lw=1.
            color_coord='1.0',
        )
        fig.savefig('{}/contibution_map_{}.png'.format(plot_dir, key))
    
    fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
        skymap=skymaps_all_bkg[key],
        label=label + ' [Bkg. Scramble]',
        cmap=cmap, norm=norm,
        insert_kwargs=dict(contour_name=key, contour_colors=['1.0', '0.7'], contour_kwargs_list=contour_kwargs_list),
        gp_kw=dict(color='0.3', alpha=0.5), #n_cb_ticks=2, gp_lw=1.
        color_coord='1.0',
    )
    fig.savefig('{}/contibution_map_{}_bkg_scramble.png'.format(plot_dir, key))
    

#### TS Contribution Map (Paper Plot)

In [ ]:
import matplotlib.patheffects as pe

# ---------------
# define colormap
# ---------------
cmap = plt.cm.inferno  
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-110, 410, 72)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# ---------------

# Define figure layout and axes
plt.rcParams.update({'font.size': 12})
fontsize_title = 19
fontsize_cb_label = 14
fontsize_text = 15
fig, axes = plt.subplots(3, 2, figsize=(12, 3), sharex=True, sharey=True)

plot_contours = True
color_coord = '1.0'
smearing_deg = 7.
qs = [0.2, 0.5]
contour_colors = ['1.0', '0.7']
contour_ls = ['--', '-.']
path_effects = [pe.Stroke(linewidth=3, foreground='0.'), pe.Normal()]
contour_kwargs_list = [dict(path_effects=path_effects), dict(path_effects=path_effects)]

for i, key in enumerate(['pi0', 'kra5', 'kra50']):
    if key == 'pi0':
        label = r'$\pi^0$'
    elif key == 'kra5':
        label = r'$\mathrm{KRA}_\gamma^5$'
    elif key == 'kra50':
        label = r'$\mathrm{KRA}_\gamma^{50}$'
    else:
        raise ValueError()
    
    if i < 2:
        modify_xticks = False
    else:
        modify_xticks = True
        
        
    # draw data
    skymap_plotter = draw_skymap_insert(
        skymaps_all[key], fig=fig, ax_insert=axes[i, 0], label=label,
        cmap=cmap, norm=norm, 
        insert_kwargs=dict(contour_name=None, modify_xticks=modify_xticks),
    )
    skymap_plotter.cb.remove() 
    if plot_contours:
        skymap_plotter.plot_template_contour(
            key, quantiles=qs, color=contour_colors, ls=contour_ls, 
            smearing_deg=smearing_deg, kwargs_list=contour_kwargs_list,
        )
    
    # draw background scrambles
    skymap_plotter = draw_skymap_insert(
        skymaps_all_bkg[key], fig=fig, ax_insert=axes[i, 1], label=label + ' [Bkg. Scramble]',
        cmap=cmap, norm=norm, 
        insert_kwargs=dict(contour_name=None, modify_xticks=modify_xticks),
    )
    skymap_plotter.cb.remove() 
    if plot_contours:
        skymap_plotter.plot_template_contour(
            key, quantiles=qs, color=contour_colors, ls=contour_ls, 
            smearing_deg=smearing_deg, kwargs_list=contour_kwargs_list,
        )
    
    # label axis
    for ax in axes[i]:
        ax.text(np.deg2rad(-175), np.deg2rad(13.2), label, va='top', ha='left', color='1.0',
                fontsize=fontsize_text,
                path_effects=[pe.Stroke(linewidth=2, foreground='0.'), pe.Normal()])
        #ax.text(0.9, 0.12,'Galactic Coord.', ha='right', va='bottom', color=color_coord, transform=ax.transAxes)
    

# draw colorbar
cax = fig.add_axes([0.33, -0.05, 0.33, 0.03])
cb = fig.colorbar(skymap_plotter.pc, cax=cax, orientation='horizontal')
cb.set_label(r'Test-Statistic Contribution $\tau$/ sr', fontsize=fontsize_cb_label)
vmin, vmax = skymap_plotter.pc.get_clim()
cb.set_ticks(np.linspace(vmin, vmax, 9))


# draw legend
for color, ls, kwargs_i, q in zip(contour_colors, contour_ls, contour_kwargs_list, qs):
    cax.plot(np.nan, np.nan, color=color, ls=ls, label='{:3.0f}% Contour'.format(q*100), **kwargs_i)
cax.legend(bbox_to_anchor=(1.1, 1.8), loc='upper left')

axes[0, 0].set_title('Observed Data', fontsize=fontsize_title)
axes[0, 1].set_title('Background Scramble', fontsize=fontsize_title)

# add panel labels
if True:
    for ax, label in zip(list(axes[:, 0]) + list(axes[:, 1]), ['A', 'B', 'C', 'D', 'E', 'F']):
        ax.text(
            .995, 0.90, label, 
            ha='right', va='top', color='1.0', fontsize=18,
            transform=ax.transAxes,
            path_effects=[pe.Stroke(linewidth=2, foreground='0.'), pe.Normal()],
        )

fig.tight_layout()
fig.savefig('{}/contibution_map.png'.format(plot_dir), bbox_inches='tight', dpi=300)
fig.savefig('{}/contibution_map.pdf'.format(plot_dir), bbox_inches='tight', dpi=300)

axes[-1, 0].text(
    0., -2, embargo_str, 
    ha='left', va='bottom', color='red', fontsize=18,
    transform=axes[-1, 0].transAxes,
)
fig.savefig('{}/contibution_map__embargo.png'.format(plot_dir), bbox_inches='tight', dpi=300)
fig.savefig('{}/contibution_map__embargo.pdf'.format(plot_dir), bbox_inches='tight', dpi=300)

plt.rcParams.update({'font.size': 10})


In [ ]:
bounds = np.linspace(-30, 410, 72)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
        skymap=skymaps_all[key],
        label=label,
        cmap=cmap, norm=norm,
        insert_kwargs=dict(contour_name=key, contour_colors=['1.0', '0.7'], contour_kwargs_list=contour_kwargs_list),
        gp_kw=dict(color='0.3', alpha=0.5), #n_cb_ticks=2, gp_lw=1.
        color_coord='1.0',
    )

## MESC 7yr Plots

In [ ]:
ss_results_mesc7yr = np.load(
    '/home/mrichman/mesc7_skymap.npy',
    allow_pickle=True,
)[()]
ss_trial_mesc7yr = np.empty((4, len(ss_results_mesc7yr)))
for i in range(len(ss_results_mesc7yr)):
    ss_trial_mesc7yr[:, i] = [v for v in ss_results_mesc7yr[i]]


##### ns-Plot

In [ ]:
skymap_plotter = SkymapPlotter(skymap=ss_trial_mesc7yr[2], cmap='viridis', label='ns', vmin=0, vmax=None)
#skymap_plotter.plot_catalog()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.savefig('{}/mesc_7yr_skymap_ns.png'.format(plot_dir))
    

##### P-value Plot

In [ ]:
skymap_plotter = SkymapPlotter(
    skymap=ss_trial_mesc7yr[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, 
    gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=7, gp_lw=1.)
skymap_plotter.ax.text(0.84, 0.16,'Equatorial Coord.', ha='right', va='bottom', color='0.3', transform=skymap_plotter.ax.transAxes)
skymap_plotter.fig.savefig('{}/mesc_7yr_skymap_pvalues.png'.format(plot_dir))


##### Gamma Plot

In [ ]:
fig, skymap_plotter, skymap_plotter_insert = make_2d_gamma_insert_plot(ss_trial=ss_trial_mesc7yr)
fig.savefig('{}/mesc_7yr_skymap_gamma_combined.png'.format(plot_dir), bbox_inches="tight")


## Compute Post-Trial Factor for Stacking Searches

In [ ]:
bkg_file_dict_stacking = {
    'snr': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'snr'),
    'pwn': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'pwn'),
    'unid': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'unid'),
}

bkg_dict_stacking = {}
for key, file_path in bkg_file_dict_stacking.items():
    print('Loading background trials for template {}'.format(key))
    bkg_dict_stacking[key] = np.load(file_path, allow_pickle=True).ts
    
        

In [ ]:
for k, values in bkg_dict_stacking.items():
    print(k, len(values))

#### Plot ts distribution

In [ ]:
for key, bg in bkg_dict_stacking.items():
    bg_tsd = cy.dists.TSD(bg)
    fig, ax = plot_bkg_trials(bg_tsd)
    ts_5sig = bg_tsd.isf_nsigma(3)
    ax.axvline(
        ts_5sig, ls='--', lw=1,
        label='3-sigma TS: {:3.3f}'.format(ts_5sig), 
    )
    ax.set_title('Stacking Catalog: {}'.format(key))
    ax.set_yscale('log')
    ax.legend()
    #fig.savefig('{}/ts_dist_{}.png'.format(plot_dir, key))

#### Compute Significance

In [ ]:
p_val_dict_stacking = {}
sigma_dict_stacking = {}
max_n = 300000000
for key, bg in bkg_dict_stacking.items():
    print(key)
    bg_tsd = cy.dists.TSD(bg[:max_n])
    p_val_dict_stacking[key] = bg_tsd.sf(bg[:max_n])
    sigma_dict_stacking[key] = bg_tsd.sf_nsigma(bg[:max_n])


#### Plot Trial Correlation

In [ ]:
import matplotlib as mpl

def plot_corr_ax_stacking(ax, key1, key2, mask=None, norm=None):
    
    if mask is None:
        mask = np.ones_like(sigma_dict_stacking[key1], dtype=bool)
        
    ax.hist2d(
        sigma_dict_stacking[key1][mask], sigma_dict_stacking[key2][mask],
        bins=bins, norm=norm, cmin=1,
    )
    ax.plot(
        (bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), 
        ls='--', color='0.7', lw=3,
    )
    ax.set_xlabel('$n\cdot \sigma$ of {}'.format(key1))
    ax.set_ylabel('$n\cdot \sigma$ of {}'.format(key2))

fig, axes = plt.subplots(3, 1, figsize=(9, 9))

bins = (np.linspace(0, 6, 50), np.linspace(0, 6, 50))
norm = mpl.colors.LogNorm(vmin=1, vmax=1e5)
mask = None
plot_corr_ax_stacking(axes[0], 'snr', 'pwn', mask=mask, norm=norm)
plot_corr_ax_stacking(axes[1], 'snr', 'unid', mask=mask, norm=norm)
plot_corr_ax_stacking(axes[2], 'pwn', 'unid', mask=mask, norm=norm)
fig.tight_layout()
#fig.savefig('{}/stacking_trial_correlation.png'.format(plot_dir))


In [ ]:
corr_keys = ['snr', 'unid', 'pwn']

max_nsigma_stacking = np.max(
    np.stack([sigma_dict_stacking[k] for k in corr_keys]),
    axis=0,
)
bg_max_stacking = cy.dists.TSD(max_nsigma_stacking)

In [ ]:
from scipy import stats

nsigma_chosen = 3.39155
pval_chosen = stats.norm.sf(nsigma_chosen)
nsigma_corrected = bg_max_stacking.sf_nsigma(nsigma_chosen)

fig, ax = plt.subplots()
ax.hist(max_nsigma_stacking, bins=np.linspace(0, 6, 200), label='Correlated bkg trials')
ax.set_xlabel('Max n-sigma')
ax.set_ylabel('Number of trials')
ax.set_yscale('log')
ax.axvline(
    nsigma_chosen, ls='--', color='0.7', 
    label='Unblinded: {:3.3f}$\sigma$ | Corrected: {:3.3f}$\sigma$'.format(
        nsigma_chosen, nsigma_corrected),
)
ax.legend(loc='upper right')
#fig.savefig('{}/stacking_trial_correction_hist.png'.format(plot_dir))

pval_corrected = bg_max_stacking.sf(nsigma_chosen)
print('Correcting for: {}'.format(corr_keys))
print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Pre-trial p-value of: {}'.format(pval_chosen))
print('Post-trial correlated n-sigma: {} | factor: {}'.format(nsigma_corrected, pval_corrected/pval_chosen))
print('Post-trial correlated p-value: {} | factor: {}'.format(pval_corrected, pval_corrected/pval_chosen))
print('Post-trial conservative n-sigma: {} | factor: {}'.format(stats.norm.isf(pval_chosen * len(corr_keys)), len(corr_keys)))
print('Post-trial conservative p-value: {} | factor: {}'.format(pval_chosen * len(corr_keys), pval_chosen * len(corr_keys)/pval_chosen))



In [ ]:
nsigma_chosen = 3.39155
#nsigma_chosen = 3.24127
pval_chosen = stats.norm.sf(nsigma_chosen)
pval_conservative = pval_chosen * 3
nsigma_conservative = stats.norm.isf(pval_conservative)

print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Post-trial conservative: {} | factor: {}'.format(nsigma_conservative, pval_conservative/pval_chosen))


## Visualize Scans of Pi^0 injections

In [ ]:
for seed in [5, 10]:
    injection_file = '/data/user/ssclafani/data/pi0_scan_seed_{}_ninj_675.npy'.format(seed)
    inj_trial = np.load(injection_file, allow_pickle=True)[()]
    # -------------------------------------
    # Compute skymap nsigma with boundaries
    # -------------------------------------
    ss_nsigmas =  stats.norm.isf(10**-inj_trial[0])

    min_val = -0.52
    mask = ss_nsigmas <= min_val
    ss_nsigmas[mask] = min_val

    mask_pole = ss_trial[3] == 0.
    ss_nsigmas[mask_pole] = np.nan

    # -------------
    # norm and cmap
    # -------------
    cmap = plt.cm.RdBu_r  # define the colormap
    N_colors = cmap.N // 2
    print('N_colors', N_colors)
    cmaplist = [cmap(i) for i in range(N_colors, cmap.N)]
    cmaplist[0] = (1., 1., 1., 1.) # force first bin to be completely white

    # create the new map
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.linspace(0, 4, 9)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    color_box = '0.7'
    #color_box = np.array([64, 73, 255]) / 255.
    fig, skymap_plotter, skymap_plotter_insert = make_skymap_insert_plot(
        skymap=ss_nsigmas, cmap=cmap, norm=norm, label='Significance: $n\cdot \sigma$', color_box=color_box)
    #skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
    #skymap_plotter_insert.plot_hotspots(marker='x', color='0.8', s=20)

    #skymap_plotter.ax.legend(bbox_to_anchor=(0.98, 0.15), loc='upper right')
    skymap_plotter_insert.ax.legend(bbox_to_anchor=(1.24, -0.1), loc='lower right') # 1.2, -0.1

    fig.savefig('{}/pi0_injections_skymap_nsigma_combined_seed{}.png'.format(plot_dir, seed))


In [ ]:
skymap_plotter = SkymapPlotter(skymap=inj_trial[2], cmap='viridis', label='ns', vmin=0, vmax=None)

## Scratch Area

In [ ]:
from scipy import stats

p_values = [2.84e-5, 1.26e-6]
print(stats.norm().isf(2.84e-5*1))

ts = -2 * np.sum(np.log(np.array(p_values)*3))

r = stats.chi2(df=len(p_values)).sf(ts)
n_sigma = stats.norm().isf(r)
print(ts, r, n_sigma)

In [ ]:
stack_dict = {}
for key in ['snr', 'pwn', 'unid']: 
    f_path = os.path.join(
        cg.base_dir, 
        'stacking/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    if os.path.exists(f_path):
        stack_dict[key] = np.load(f_path)
stack_dict

In [ ]:
def get_dist_to_shower_max(ref_energy, eps=1e-6):
    """Get cascade extension for an EM cascade

    Parameters
    ----------
    ref_energy : float or np.ndarray of floats
        Energy of cascade in GeV.
    eps : float, optional
        Small constant float.

    Returns
    -------
    float or array_like
        The distance from the cascade vertex to the average shower maximum.
    """

    # Radiation length in meters, assuming an ice density of 0.9216 g/cm^3
    l_rad = (0.358/0.9216)  # in meter

    """
    Parameters taken from I3SimConstants (for particle e-):
    https://code.icecube.wisc.edu/projects/icecube/browser/IceCube/
    meta-projects/combo/trunk/sim-services/private/
    sim-services/I3SimConstants.cxx
    """
    a = 2.01849 + 0.63176 * np.log(ref_energy + eps)
    b = l_rad/0.63207

    # Mode of the gamma distribution gamma_dist(a, b) is: (a-1.)/b
    length_to_maximum = np.clip(((a-1.)/b)*l_rad, 0., float('inf'))
    return length_to_maximum

for energy in np.linspace(3, 5):
    print(10**energy, get_dist_to_shower_max(10**energy))

In [ ]:
def weighted_quantile(x, weights, quantile):

    if weights is None:
        weights = np.ones_like(x)

    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    weights_sorted = weights[sorted_indices]
    cum_weights = np.cumsum(weights_sorted) / np.sum(weights)
    idx = np.searchsorted(cum_weights, quantile)
    return x_sorted[idx]

def get_flux(tr):
    for key, item in tr.llh_kw['conf'].items():
        if 'flux' == key:
            return item
        if isinstance(item, dict):
            for key_sub, item_sub in item.items():
                if key_sub == 'flux':
                    return item_sub
    return None

flux = get_flux(tr_dict['pi0'])
weights = a.sig.oneweight * flux(a.sig.true_energy)
weighted_quantile(a.sig.true_energy, weights=weights, quantile=np.r_[0.0005, 0.9995])

In [ ]:
for ipx in np.argsort(ss_nsigmas)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print(np.rad2deg(dec), np.rad2deg(phi))

In [ ]:
np.argsort(ss_nsigmas_cpy)[::-1]

In [ ]:
ra_deg, dec_deg = fermi_sources['Crab']
theta = np.pi/2. - np.deg2rad(dec_deg)
phi = np.deg2rad(ra_deg)
mask_crab = hp.query_disc(
    nside=128, 
    vec=hp.dir2vec(theta=theta, phi=phi), 
    radius=np.deg2rad(10),
)
ipx_crab = hp.ang2pix(128, theta=theta, phi=phi)



ss_nsigmas_cpy = np.zeros_like(ss_nsigmas)
ss_nsigmas_cpy[mask_crab] = np.array(ss_nsigmas[ipix_crab])
fig, ax, sp = plot_skymap(ss_nsigmas_cpy, label=r'$n\cdot \sigma$')

for ipx in np.argsort(ss_nsigmas_cpy)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print('{:3.3f} {:3.3f} | {:3.3f} $n\cdot \sigma$'.format(
        np.rad2deg(phi), np.rad2deg(dec), ss_nsigmas_cpy[ipx]))
print(fermi_sources['Crab'], ss_nsigmas[ipx_crab])

#### Collect event ids for 10 most energetic events

In [ ]:
idx = np.argsort(a.bg_data.energy)[::-1][:10]
print(a.bg_data.energy[idx])

import pandas as pd

df = pd.DataFrame({
    'run': a.bg_data.run[idx],
    'event': a.bg_data.event[idx],
    'subevent': a.bg_data.subevent[idx],
})
df.to_pickle('{}/top_10E_events.pickle'.format(plot_dir))
df


In [ ]:
for gamma in [2., 2.5, 3.]:
    plt.hist(a.sig.true_energy, bins=np.logspace(1, 7, 100), histtype='step', density=True,
             weights=a.sig.oneweight * a.sig.true_energy**-gamma, label='gamma: {:3.3f}'.format(gamma))
plt.axvline(100, color='0.7', ls=':')
plt.axvline(500, color='0.7', ls='--')
plt.axvline(1000, color='0.7', ls='-')
plt.xscale('log')
plt.yscale('log')
plt.legend()


In [ ]:
from scipy import stats
s = 1.
stats.norm.sf(s), stats.norm.sf(s)*109, 1./109